# Building a MongoDB-Powered AML Solution
The implementation of these three core features—Intelligent Entity Resolution, Network Analysis & Visualization, and Real-Time Risk Scoring creates a powerful foundation for a modern AML system built on MongoDB. Together, they address the fundamental challenges facing financial institutions in the fight against money laundering:
* Who are we dealing with? (Entity Resolution)
* Who do they know and work with? (Network Analysis)
* How risky are they and why? (Real-Time Risk Scoring)

In [ ]:
!pip install pymongo faker dnspython boto3 awscli numpy -q

import pymongo
from faker import Faker
import random
from datetime import datetime, timedelta
from bson import ObjectId
import uuid # For more unique IDs if needed
import boto3
from botocore.config import Config
from datetime import datetime, timedelta, timezone
import numpy as np
import time
import json

In [ ]:
# --- Configuration ---
MONGODB_URI = "ENTER URI"
DB_NAME = "threatsight360"

# Add your AWS credentials and region
aws_access_key = ""
aws_secret_key = ""
aws_region = "us-east-1"  # Change to your preferred region
EMBEDDING_MODEL_ID = "amazon.titan-embed-text-v1"
EMBEDDING_DIMENSIONS = 1536

In [ ]:
# fake = Faker() # Your original
fake = Faker(['en_US', 'en_CA', 'en_GB', 'de_DE', 'fr_FR']) # New: Add multiple locales
Faker.seed(0) # For reproducibility

In [ ]:
# Configure AWS Session
boto3_config = Config(
    region_name=aws_region,
    signature_version='v4',
    retries={
        'max_attempts': 3,
        'mode': 'standard'
    }
)

# Initialize Bedrock Runtime client
try:
    bedrock_runtime = boto3.client(
        service_name='bedrock-runtime',
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key,
        config=boto3_config
    )
    print("AWS Bedrock client initialized successfully")
except Exception as e:
    print(f"Warning: AWS Bedrock client initialization failed: {e}")
    print("Using fallback random embeddings - for demo purposes only")
    bedrock_runtime = None

AWS Bedrock client initialized successfully


In [ ]:
# --- Embedding Generation Function ---
def get_embedding_from_bedrock_or_fallback(text_to_embed, bedrock_client, model_id=EMBEDDING_MODEL_ID, dimensions=EMBEDDING_DIMENSIONS):
    if not bedrock_client:
        # print(f"Debug: Bedrock client None. Fallback random embedding for: '{text_to_embed[:50]}...'")
        return np.random.rand(dimensions).tolist()
    try:
        payload = {"inputText": text_to_embed}
        body = json.dumps(payload)
        response = bedrock_client.invoke_model(
            body=body, modelId=model_id, accept='application/json', contentType='application/json'
        )
        response_body = json.loads(response.get('body').read())
        embedding = response_body.get('embedding')
        if embedding and len(embedding) == dimensions:
            return embedding
    except Exception as e:
        print(f"Error generating embedding with Bedrock for '{text_to_embed[:50]}...': {e}")

In [ ]:
# --- MongoDB Connection ---
try:
    client = pymongo.MongoClient(MONGODB_URI)
    db = client[DB_NAME]
    db.command('ping')
    print(f"Successfully connected to MongoDB: {DB_NAME}")
except Exception as e:
    print(f"Error connecting to MongoDB: {e}")
    exit()



# 1. `entities` Collection

*   **Schema:** (Your existing schema is very good. We'll highlight key parts for generation.)
    *   `_id`: `ObjectId()`
    *   `entityId`: String (e.g., "E" + random numbers/letters) - **Unique**
    *   `entityType`: String ("individual", "organization")
    *   `status`: String ("active", "inactive", "under_review")
    *   `sourceSystem`: String (e.g., "core_banking", "onboarding_portal", "crm")
    *   `createdAt`: `ISODate()`
    *   `updatedAt`: `ISODate()`
    *   `name`:
        *   `full`: String
        *   `structured`: { `first`, `middle`, `last` } (for individuals) or `legalName` (for orgs)
        *   `aliases`: Array of Strings
        *   `nameComponents`: Array of Strings (lowercase tokens)
    *   `dateOfBirth`: String "YYYY-MM-DD" (for individuals) / `incorporationDate` for orgs
    *   `placeOfBirth`: String (for individuals) / `jurisdictionOfIncorporation` for orgs
    *   `gender`: String (for individuals)
    *   `nationality`: Array of Strings (for individuals)
    *   `residency`: String (country code)
    *   `addresses`: Array of Objects (your address schema is great)
        *   `type`: "residential", "business", "mailing"
        *   `primary`: Boolean
        *   `full`: String
        *   `structured`: { `street`, `city`, `state`, `postalCode`, `country` }
        *   `coordinates`: [lng, lat] (optional, but cool for geo-queries if you go there)
        *   `validFrom`, `validTo`: `ISODate()`
        *   `verified`, `verificationMethod`, `verificationDate`
    *   `contactInfo`: Array of Objects (your contactInfo schema is good)
        *   `type`: "email", "phone"
        *   `value`: String
        *   `primary`: Boolean
        *   `verified`, `verificationDate`
    *   `identifiers`: Array of Objects (your identifiers schema is good)
        *   `type`: "ssn", "passport", "drivers_license", "tin" (for orgs)
        *   `value`: String
        *   `country`: String
        *   `issueDate`, `expiryDate`: `ISODate()`
        *   `verified`, `verificationMethod`, `verificationDate`
    *   `resolution`: Object (Crucial for Feature 1)
        *   `status`: "unresolved", "resolved", "under_review" (default to "unresolved" or "resolved" if it's a master)
        *   `masterEntityId`: String (points to self if it's the master, or to the master entityId)
        *   `confidence`: Number (0-1)
        *   `linkedEntities`: Array of Objects
            *   `entityId`: String (ID of the linked entity)
            *   `linkType`: "potential_duplicate", "confirmed_match", "shared_address_link", etc.
            *   `confidence`: Number
            *   `matchedAttributes`: Array of Strings
            *   `matchDate`: `ISODate()`
            *   `decidedBy`: "system" or "analyst_ID"
            *   `decision`: "confirmed_match", "rejected_match", "under_review"
        *   `lastReviewDate`, `reviewedBy`
    *   `riskAssessment`: Object (Crucial for Feature 3 & pKYC)
        *   `overall`: { `score`, `level`, `trend`, `lastUpdated`, `nextScheduledReview` }
        *   `components`: { `identity`, `activity`, `profile`, `external`, `network` } (each with `score`, `weight`, `factors` array)
            *   `factors`: [{ `type`, `impact`, `description`, `mitigations` }]
        *   `history`: Array of { `date`, `score`, `level`, `changeTrigger` }
        *   `metadata`: { `model`, `lastFullAssessment`, `assessmentType`, `overrides` }
    *   `watchlistMatches`: Array of Objects (Crucial for Feature 3/5)
        *   `listId`: String (e.g., "OFAC-SDN", "EU-PEP")
        *   `matchId`: String (ID from the watchlist source)
        *   `matchScore`: Number
        *   `matchDate`: `ISODate()`
        *   `status`: "under_review", "confirmed_hit", "false_positive"
    *   `customerInfo`: Object (Good for context and profile risk)
        *   `customerSince`, `segments`, `products`, `employmentStatus`, `occupation`, `employer` / `industry`, `businessType` (for orgs)
    *   **Data Tweaks for `entities`:**
        1.  **Clear Duplicates:**
            *   Entity A: "Johnathan Smith", DOB "1980-01-15", Address "123 Main St, Anytown".
            *   Entity B: "John Smith", DOB "1980-01-15", Address "123 Main St, Apt 2, Anytown". (Slight name/address variation)
            *   Entity C: "Jon Smyth", DOB "1980-01-15", Phone "+1-555-123-4567".
            *   Entity D: "Johnathan A. Smith", same DOB, same phone as C.
        2.  **Subtle/Complex Duplicates:**
            *   Entity E: "Sarah Miller", Address "456 Oak Ave", Old Passport "P123".
            *   Entity F: "Sarah Davis" (married name), Address "789 Pine Ln", New Passport "P789", Email "sarah.davis@email.com" (previously used by Sarah Miller). Link via historical data or shared non-obvious identifier.
        3.  **Organization Links:**
            *   Org G: "Global Exports Inc.", Director: "Peter Jones" (Entity H).
            *   Org I: "Import Solutions LLC", Shareholder >25%: "Peter Jones" (Entity H).
        4.  **Risk Profiles:**
            *   **Low Risk Start:** Entity J - complete KYC, stable job, low-risk country.
            *   **High Risk Start:** Entity K - PEP, operations in high-risk industry/jurisdiction.
            *   **Evolving Risk:** Entity L (starts low) - later add transactions to high-risk country, or link to a high-risk entity, or partial watchlist match.
        5.  **Watchlist Candidates:**
            *   Entity M: "Evil Villain" - direct match for a watchlist entry.
            *   Entity N: "Eva Villian" - fuzzy match for a watchlist entry.
        6.  **For Network:** Entities that share addresses, phone numbers, or are linked via ER.
        7.  **Temporal Data:** Some entities with multiple historical addresses/jobs to show the timeline.
        8.  **Incomplete Data:** Some entities with missing `dateOfBirth` or unverified identifiers to show impact on identity risk.


In [ ]:
entities_collection = db["entities"]
potential_ubo_pool = [] # Reset for each run if needed

# --- Global Storage for Generated IDs (to be used by other collection scripts) ---\n",
# This dictionary will store lists of entity_docs for easy access later\n",
generated_entity_store = {}

# --- Enhanced Helper Functions ---\n",
def generate_unique_id(prefix="E"):
    return f"{prefix}-{uuid.uuid4().hex[:10].upper()}"

def create_name_data(entity_type="individual", first_name=None, last_name=None, company_name_base=None, use_maiden_name=False):
    name_info = {}
    if entity_type == "individual":
        fn = first_name or fake.first_name()
        ln = last_name or fake.last_name()
        mn = fake.first_name() if random.random() < 0.4 else "" # Increased middle name chance
        full = f"{fn} {mn} {ln}".replace("  ", " ").strip()

        aliases = []
        if mn: aliases.append(f"{fn} {ln}") # Alias without middle name
        aliases.append(f"{fn[0]}. {ln}")
        if random.random() < 0.2: aliases.append(f"{fn} {ln[0]}.")
        if random.random() < 0.15: aliases.append(fake.first_name() + " " + ln) # Different first name, same last
        if use_maiden_name and random.random() < 0.5 : # More common if flagged
             maiden_name = fake.last_name()
             aliases.append(f"{fn} {maiden_name}")
             aliases.append(f"{fn} {mn} {maiden_name}".replace("  ", " ").strip())


        name_info = {
            "full": full, "structured": {"first": fn, "middle": mn, "last": ln},
            "aliases": list(set(aliases))[:3], # Limit number of aliases
            "nameComponents": [n.lower() for n in full.split()]
        }
    else: # organization
        base = company_name_base or fake.bs().replace(" ", "-").split('-')[0].capitalize()
        if random.random() < 0.2: # Add more varied base names
            base = random.choice([fake.word().capitalize() + fake.word().capitalize(), fake.last_name() + " Holdings"])

        suffix = random.choice([
            "Inc.", "Ltd.", "LLC", "Corp.", "Group", "Solutions", "Global", "Ventures", "Holdings",
            "Enterprises", "International", "Partners", "Associates", "Industries", "Logistics", "Trading Co."
        ])
        full = f"{base} {suffix}"

        aliases = []
        if random.random() < 0.3: aliases.append(f"{base} {random.choice(['Enterprises', 'Services', 'Worldwide', 'Group'])}")
        if random.random() < 0.2: aliases.append(f"{base.split()[0] if ' ' in base else base[:5]} {suffix}") # Abbreviated base

        name_info = {
            "full": full, "structured": {"legalName": full}, "aliases": list(set(aliases))[:2],
            "nameComponents": [n.lower() for n in full.split()]
        }
    return name_info

def create_address_data(primary=True, country_code=None, city=None, risk_level=None, structured=None, full=None, verified_status=None, address_type=None):
    # Expanded country lists, especially high-risk
    countries = {
        "low": [("US", "United States"), ("CA", "Canada"), ("DE", "Germany"), ("FR", "France"), ("AU", "Australia"), ("JP", "Japan"), ("NZ", "New Zealand"), ("NO", "Norway")],
        "medium": [("GB", "United Kingdom"), ("CH", "Switzerland"), ("AE", "United Arab Emirates"), ("SG", "Singapore"), ("HK", "Hong Kong"), ("BR", "Brazil"), ("ZA", "South Africa"), ("CN", "China")],
        "high": [
            ("SY", "Syria"), ("KP", "North Korea"), ("VE", "Venezuela"), ("KY", "Cayman Islands"), ("PA", "Panama"),
            ("IR", "Iran"), ("AF", "Afghanistan"), ("SO", "Somalia"), ("YE", "Yemen"), ("LY", "Libya"), ("IQ", "Iraq"),
            ("VU", "Vanuatu"), ("BS", "Bahamas"), ("CY", "Cyprus"), ("MT", "Malta"), ("TC", "Turks and Caicos Islands")
        ]
    }
    country_map_direct = {code: name for k_list in countries.values() for code, name in k_list}


    if structured and structured.get("country"):
        chosen_country_code = structured["country"]
        chosen_country_name = country_map_direct.get(chosen_country_code, fake.country())
    elif country_code:
        chosen_country_code = country_code
        chosen_country_name = country_map_direct.get(country_code, fake.country())
    else:
        chosen_risk = risk_level or random.choice(list(countries.keys()))
        # Ensure chosen_risk is valid if passed explicitly, otherwise default
        if chosen_risk not in countries: chosen_risk = random.choice(list(countries.keys()))
        chosen_country_code, chosen_country_name = random.choice(countries[chosen_risk])


    addr_struct = structured
    full_addr_override = full

    if not addr_struct:
        addr_city_gen = city or fake.city()
        addr_street_gen = fake.street_address()
        # More robust state/province generation
        if chosen_country_code == "US": addr_state_gen = fake.state_abbr()
        elif chosen_country_code == "CA": addr_state_gen = fake.province_abbr()
        elif chosen_country_code == "AU": addr_state_gen = fake.state_abbr() # Faker has AU states
        else: addr_state_gen = fake.state() if random.random() < 0.3 else "" # Generic state for others, or none

        addr_postal_gen = fake.zipcode() if chosen_country_code == "US" else fake.postcode()
        addr_struct = {
            "street": addr_street_gen, "city": addr_city_gen, "state": addr_state_gen,
            "postalCode": addr_postal_gen, "country": chosen_country_code
        }

    if not full_addr_override:
        s = addr_struct
        full_addr_override = f"{s['street']}, {s['city']}{', '+s['state'] if s['state'] else ''}, {s['postalCode']}, {chosen_country_name}"

    if verified_status is not None:
        is_verified = verified_status
    else:
        is_verified = random.random() > (0.4 if risk_level == "high" else 0.2) # Less likely verified if high risk address

    addr_type_choices = ["residential", "business", "mailing", "registered_office", "previous", "care_of"]
    addr = {
        "type": address_type or random.choice(addr_type_choices),
        "primary": primary,
        "full": full_addr_override,
        "structured": addr_struct,
        "coordinates": [float(fake.longitude()), float(fake.latitude())] if random.random() < 0.6 else None,
        "validFrom": fake.date_time_between(start_date="-10y", end_date="-6m", tzinfo=timezone.utc),
        "validTo": None,
        "verified": is_verified,
        "verificationMethod": random.choice([
            "utility_bill", "electronic_idv", "site_visit", "lease_agreement", "public_record", "correspondence"
            ]) if is_verified else None,
        "verificationDate": fake.date_time_this_year(tzinfo=timezone.utc) if is_verified else None
    }
    return addr

def create_contact_data(primary=True, contact_type_override=None, value_override=None):
    contact_type = contact_type_override or random.choice(["email", "phone_mobile", "phone_landline", "fax", "social_media_handle"])
    if value_override:
        value = value_override
    elif "email" in contact_type: value = fake.email()
    elif "social_media" in contact_type: value = "@" + fake.user_name()
    else: value = fake.phone_number()

    is_verified = random.random() > 0.35
    return {"type": contact_type, "value": value, "primary": primary, "verified": is_verified,
            "verificationDate": fake.date_time_this_year(tzinfo=timezone.utc) if is_verified else None}

def create_identifier_data(entity_type="individual", country_code="US", id_type=None): # Added id_type override
    actual_id_type, value = "", ""
    issue_country = country_code
    if entity_type == "individual":
        id_types_available = ["passport", "national_id", "drivers_license", "ssn", "tax_id", "voter_id", "health_card_id"]
        actual_id_type = id_type or random.choice(id_types_available)

        if actual_id_type == "passport": value, issue_country = fake.unique.bothify(text="??#########", letters="ABCDEFGHIJKLMNOPQRSTUVWXYZ"), random.choice([country_code, "GB", "CA", "DE", "FR", "AU"]) if random.random() < 0.7 else fake.country_code()
        elif actual_id_type == "ssn" and country_code == "US": value = fake.ssn()
        elif actual_id_type == "national_id": value = fake.unique.bothify(text="ID-############??")
        elif actual_id_type == "tax_id": value = fake.unique.bothify(text="TAXID-??########")
        else: value = fake.unique.bothify(text="??-?#########", letters="ABCDEF")
    else: # organization
        id_types_available = ["registration_no", "tax_id_number", "vat_id", "lei_code", "duns_number"]
        actual_id_type = id_type or random.choice(id_types_available)
        if actual_id_type == "tax_id_number": value = fake.unique.bothify(text="TIN##-#######")
        elif actual_id_type == "vat_id": value = f"{country_code}{fake.unique.bothify(text='#########')}"
        elif actual_id_type == "lei_code": value = fake.unique.bothify(text="##################??", letters="0123456789ABCDEFGHJKLMNPQRSTUVWXYZ") # 20 chars
        else: value = fake.unique.bothify(text="REG-##########??")

    is_verified = random.random() > (0.3 if actual_id_type == "ssn" else 0.15) # SSNs harder to verify synthetically
    return {"type": actual_id_type, "value": value, "country": issue_country,
            "issueDate": fake.date_time_between(start_date="-12y", end_date="-3m", tzinfo=timezone.utc),
            "expiryDate": fake.date_time_between(start_date="+6m", end_date="+10y", tzinfo=timezone.utc) if actual_id_type in ["passport", "drivers_license", "lei_code"] else None,
            "verified": is_verified,
            "verificationMethod": random.choice(["document_scan", "issuing_authority_check", "database_lookup", "api_validation"]) if is_verified else None,
            "verificationDate": fake.date_time_this_decade(tzinfo=timezone.utc) if is_verified else None}

# Function to generate UBO data for organizations
def generate_ubo_data(num_ubos=0, existing_individual_pool=None):
    ubos = []
    if num_ubos == 0:
        num_ubos = random.choices([0,1,2,3], weights=[0.2, 0.4, 0.3, 0.1], k=1)[0]

    for _ in range(num_ubos):
        ubo_type = random.choices(["individual", "corporate"], weights=[0.85, 0.15], k=1)[0]
        name_data = create_name_data(entity_type="individual" if ubo_type == "individual" else "organization")

        ubo_entry = {
            "name": name_data["full"],
            "entityType": ubo_type,
            "nationality": fake.country_code() if ubo_type == "individual" else None,
            "countryOfIncorporation": fake.country_code() if ubo_type == "corporate" else None,
            "percentageOwnership": round(random.uniform(5.0, 95.0), 2), # More varied ownership
            "controlType": random.choice(["direct_ownership", "indirect_ownership", "voting_rights", "board_control", "other_influence"]),
            "identification": { # Simplified ID for UBO for now
                "type": "passport" if ubo_type == "individual" else "registration_no",
                "value": fake.unique.bothify(text="UBO-ID-????####")
            } if random.random() < 0.7 else None,
            "linkedEntityId": None # Placeholder for potential linking later
        }
        # Optionally link to an existing individual from a pool
        if ubo_type == "individual" and existing_individual_pool and random.random() < 0.2:
            chosen_ind = random.choice(existing_individual_pool)
            ubo_entry["name"] = chosen_ind["name"]["full"]
            ubo_entry["nationality"] = chosen_ind["nationality"][0] if chosen_ind.get("nationality") else fake.country_code()
            ubo_entry["linkedEntityId"] = chosen_ind["entityId"]
            ubo_entry["percentageOwnership"] = round(random.uniform(25.0, 75.0), 2) # Higher if linked to specific person

        ubos.append(ubo_entry)
    return ubos


def calculate_detailed_risk(entity_doc):
    risk_factors_details = {"identity": [], "profile": [], "activity": [], "external": [], "network": []}
    # Base scores - can adjust these defaults
    identity_score, profile_score, activity_score, external_score, network_score = 10, 10, 20, 0, 10

    # --- Identity Risk Factors ---
    if not entity_doc.get("dateOfBirth") and entity_doc["entityType"] == "individual":
        identity_score += 25; risk_factors_details["identity"].append({"type": "missing_dob", "impact": 25, "description": "Date of birth is missing."})

    unverified_ids_count = sum(1 for i in entity_doc.get("identifiers", []) if not i.get("verified"))
    if unverified_ids_count > 0:
        id_impact = unverified_ids_count * 15
        identity_score += id_impact; risk_factors_details["identity"].append({"type": "unverified_identifiers", "impact": id_impact, "description": f"{unverified_ids_count} unverified identifier(s)."})
    if len(entity_doc.get("identifiers", [])) == 0:
        identity_score += 30; risk_factors_details["identity"].append({"type": "no_identifiers", "impact": 30, "description": "No identifiers provided."})

    if len(entity_doc.get("name",{}).get("aliases",[])) > 2:
        identity_score += 10; risk_factors_details["identity"].append({"type": "multiple_aliases", "impact": 10, "description": "Multiple aliases present."})

    num_unverified_contacts = sum(1 for c in entity_doc.get("contactInfo", []) if not c.get("verified"))
    if num_unverified_contacts > 1 :
        identity_score += 5 * num_unverified_contacts; risk_factors_details["identity"].append({"type": "unverified_contact", "impact": 5 * num_unverified_contacts, "description": f"{num_unverified_contacts} unverified contact methods."})

    # --- Profile Risk Factors ---
    cust_info = entity_doc.get("customerInfo", {})
    if entity_doc["entityType"] == "individual":
        if cust_info.get("employmentStatus") == "unemployed":
            profile_score += 20; risk_factors_details["profile"].append({"type": "unemployed", "impact": 20, "description": "Individual is unemployed."})
        occupation_value = cust_info.get("occupation") # Get the value, could be None
        occupation = str(occupation_value).lower() if occupation_value is not None else "" # Convert to string then lower, or default to ""
        # Expanded high-risk industries/occupations
        high_risk_occupations = [
            "casino dealer", "arms dealer", "pawnbroker", "money transmitter", "virtual currency trader",
            "dealer in precious metals/stones", "political consultant", "lobbyist"
        ]
        if any(hr_occ in occupation for hr_occ in high_risk_occupations):
             profile_score += 30; risk_factors_details["profile"].append({"type": "high_risk_occupation", "impact": 30, "description": f"Occupation '{occupation}' is high-risk."})
    else: # Organization
        industry = cust_info.get("industry", "").lower()
        high_risk_industries = [
            "casinos", "arms manufacturing", "virtual currency exchange", "money service business",
            "shell company formation", "offshore banking", "art and antiquities", "real estate (high value cash)"
        ]
        if any(hr_ind in industry for hr_ind in high_risk_industries):
            profile_score += 35; risk_factors_details["profile"].append({"type": "high_risk_industry_org", "impact": 35, "description": f"Industry '{industry}' is high-risk."})

        if cust_info.get("businessType") == "shell_company_suspected": # Need to set this in scenario
             profile_score += 40; risk_factors_details["profile"].append({"type": "suspected_shell_company", "impact": 40, "description": "Business type and characteristics suggest shell company."})

        if len(entity_doc.get("uboInfo", [])) == 0 and random.random() < 0.3 : # No UBO info for an org is a risk
             profile_score += 25; risk_factors_details["profile"].append({"type": "missing_ubo_data", "impact": 25, "description": "Ultimate Beneficial Ownership information is missing or incomplete."})


    high_risk_address_countries = ["KY", "PA", "SY", "KP", "VE", "IR", "AF", "SO", "YE", "VU", "BS", "CY", "MT", "TC"]
    num_hr_addresses = 0
    for addr in entity_doc.get("addresses", []):
        if addr.get("structured", {}).get("country") in high_risk_address_countries:
            num_hr_addresses +=1
            if addr.get("primary"): # Primary address in HR country is higher risk
                profile_score += 30; risk_factors_details["profile"].append({"type": "primary_high_risk_jurisdiction", "impact": 30, "description": "Primary address in high-risk jurisdiction."})
            else:
                 profile_score += 15; risk_factors_details["profile"].append({"type": "other_high_risk_jurisdiction", "impact": 15, "description": "Non-primary address in high-risk jurisdiction."})
            # break # one is enough for this factor usually, but could count them # Decided to allow multiple factors
    if num_hr_addresses > 1:
        profile_score += 10; risk_factors_details["profile"].append({"type": "multiple_high_risk_jurisdictions", "impact": 10, "description": "Presence in multiple high-risk jurisdictions."})

    if entity_doc.get("sourceSystem") == "manual_entry_suspicious": # Need to set this in scenario
        profile_score += 15; risk_factors_details["profile"].append({"type": "suspicious_onboarding", "impact": 15, "description": "Onboarded via a potentially suspicious manual process."})


    # --- External Risk Factors (mainly watchlist) ---
    if entity_doc.get("watchlistMatches"):
        for match in entity_doc["watchlistMatches"]:
            if match.get("status") == "confirmed_hit":
                external_score += 70; risk_factors_details["external"].append({"type": "confirmed_watchlist_hit", "impact": 70, "description": f"Confirmed match on {match.get('listId')}."})
            elif match.get("status") == "under_review":
                external_score += 35; risk_factors_details["external"].append({"type": "pending_watchlist_match", "impact": 35, "description": f"Potential match on {match.get('listId')} under review."})
            elif match.get("status") == "fuzzy_match_high_confidence":
                external_score += 25; risk_factors_details["external"].append({"type": "fuzzy_watchlist_match_strong", "impact": 25, "description": f"Strong fuzzy match on {match.get('listId')}."})

    # Activity and Network scores would typically be updated by other processes (transaction monitoring, graph analysis)
    # For initial seeding, they can be low or have some baseline if certain profile elements imply network risk.
    if len(entity_doc.get("resolution", {}).get("linkedEntities", [])) > 3:
        network_score += 10; risk_factors_details["network"].append({"type":"multiple_linked_entities", "impact":10, "description":"Connected to multiple other entities."})


    # Weights for each component
    weights = {"identity": 0.25, "profile": 0.35, "activity": 0.20, "external": 0.15, "network": 0.05}

    # Ensure scores are within 0-100 before weighting
    identity_score = min(max(identity_score, 0), 100)
    profile_score = min(max(profile_score, 0), 100)
    activity_score = min(max(activity_score, 0), 100)
    external_score = min(max(external_score, 0), 100)
    network_score = min(max(network_score, 0), 100)

    overall_score = (identity_score * weights["identity"]) + \
                    (profile_score * weights["profile"]) + \
                    (activity_score * weights["activity"]) + \
                    (external_score * weights["external"]) + \
                    (network_score * weights["network"])

    overall_score = min(max(int(overall_score), 0), 100)
    level = "high" if overall_score > 70 else "medium" if overall_score > 40 else "low" # Adjusted thresholds

    components_data = {
        "identity": {"score": identity_score, "weight": weights["identity"], "factors": risk_factors_details["identity"]},
        "profile": {"score": profile_score, "weight": weights["profile"], "factors": risk_factors_details["profile"]},
        "activity": {"score": activity_score, "weight": weights["activity"], "factors": risk_factors_details["activity"]},
        "external": {"score": external_score, "weight": weights["external"], "factors": risk_factors_details["external"]},
        "network": {"score": network_score, "weight": weights["network"], "factors": risk_factors_details["network"]},
    }
    return overall_score, level, components_data


def generate_entity_template(entity_type, scenario_key="generic", existing_individual_pool=None, **kwargs):
    created_at = fake.date_time_between(start_date="-7y", end_date="-1d", tzinfo=timezone.utc) # Ensure not too recent
    updated_at = fake.date_time_between(start_date=created_at, end_date="now", tzinfo=timezone.utc)

    name_data = create_name_data(
        entity_type,
        kwargs.get("first_name"),
        kwargs.get("last_name"),
        kwargs.get("company_name_base"),
        kwargs.get("use_maiden_name_flag", False)
    )

    addresses = []
    # Primary Address
    primary_addr_details = {"primary": True, "country_code": kwargs.get("country_code"), "city": kwargs.get("city"), "risk_level": kwargs.get("address_risk")}
    if "primary_address_struct" in kwargs: primary_addr_details["structured"] = kwargs["primary_address_struct"]
    if "primary_address_full" in kwargs: primary_addr_details["full"] = kwargs["primary_address_full"]
    if "primary_address_verified" in kwargs: primary_addr_details["verified_status"] = kwargs["primary_address_verified"]
    addresses.append(create_address_data(**primary_addr_details))

    # Optional: More addresses (historical, secondary)
    num_other_addresses = random.choices([0,1,2], weights=[0.5, 0.3, 0.2], k=1)[0]
    for i in range(num_other_addresses):
        addr_risk = random.choice(["low", "medium", "high"]) if not kwargs.get("force_addr_risk") else kwargs.get("force_addr_risk")
        past_address = create_address_data(
            primary=False,
            country_code=kwargs.get(f"past_country_code_{i}"),
            risk_level=addr_risk, # Vary risk of past addresses
            address_type=random.choice(["previous", "mailing", "business"])
        )
        past_address["validTo"] = past_address["validFrom"] + timedelta(days=random.randint(180, 1500))
        if past_address["validTo"] > datetime.now(timezone.utc): # ensure past is past
            past_address["validTo"] = fake.date_time_between(start_date=past_address["validFrom"], end_date="-1d", tzinfo=timezone.utc)
        addresses.append(past_address)

    current_country = addresses[0]["structured"]["country"] # Use primary address country for identifiers

    identifiers = []
    num_identifiers = random.choices([1,2,3], weights=[0.4, 0.4, 0.2], k=1)[0]
    if "num_ids" in kwargs: num_identifiers = kwargs["num_ids"]
    if kwargs.get("force_no_ids", False): num_identifiers = 0

    for _ in range(num_identifiers):
        identifiers.append(create_identifier_data(entity_type, country_code=current_country))
    if "specific_ids" in kwargs: # Allow adding very specific IDs for scenarios
        identifiers.extend(kwargs["specific_ids"])


    entity_doc = {
        "_id": ObjectId(), "entityId": kwargs.get("entityId", generate_unique_id("C" if entity_type == "individual" else "O")),
        "scenarioKey": scenario_key, "entityType": entity_type, "status": kwargs.get("status", "active"),
        "sourceSystem": kwargs.get("source_system_override", random.choice([
            "onboarding_v3_digital", "crm_salesforce", "legacy_mainframe_sysX",
            "partner_api_acme", "manual_entry_branch", "third_party_data_enrichment"
            ])),
        "createdAt": created_at, "updatedAt": updated_at, "name": name_data, "addresses": addresses,
        "contactInfo": [create_contact_data(primary=True, contact_type_override=kwargs.get("primary_contact_type"), value_override=kwargs.get("primary_contact_value"))] + \
                       ([create_contact_data(primary=False)] if random.random() < 0.6 else []) + \
                       ([create_contact_data(primary=False)] if random.random() < 0.3 else []), # More contacts
        "identifiers": identifiers,
        "resolution": {"status": "unresolved", "masterEntityId": None, "confidence": 0.0, "linkedEntities": [], "lastReviewDate": None, "reviewedBy": None},
        "watchlistMatches": kwargs.get("watchlistMatches", []),
        "customerInfo": {
            "customerSince": created_at - timedelta(days=random.randint(90, 365*5)), # Wider range for customer since
            "segments": random.sample([
                "retail_banking", "private_wealth_management", "sme_lending", "corporate_banking_large",
                "institutional_investor", "correspondent_banking", "mass_affluent", "student_accounts"
                ], k=random.randint(1,3)),
            "products": random.sample([
                "checking_account_basic", "savings_plus_high_yield", "global_platinum_credit_card",
                "commercial_real_estate_loan", "prime_residential_mortgage", "managed_investment_portfolio_aggressive",
                "trade_finance_lc", "foreign_exchange_services", "digital_wallet_services", "business_overdraft_facility"
                ], k=random.randint(1,5)), # More products
            "notes": fake.paragraph(nb_sentences=random.randint(1,2)) if random.random() < 0.2 else None
        }
    }
    cust_info_ref = entity_doc["customerInfo"] # Create a reference for easier use in summary

    if entity_type == "individual":
        entity_doc.update({
            "dateOfBirth": kwargs.get("dob_str") if kwargs.get("dob_str", "NOT_SET") != "NOT_SET" else fake.date_of_birth(minimum_age=18, maximum_age=95).strftime("%Y-%m-%d"),
            "placeOfBirth": f"{fake.city()}, {fake.country()}",
            "gender": random.choice(["male", "female", "non_binary", "other", "undisclosed"]),
            "nationality": [addresses[0]["structured"]["country"], fake.country_code()] if random.random() < 0.25 else [addresses[0]["structured"]["country"]],
            "residency": addresses[0]["structured"]["country"]
        })
        cust_info_ref.update({ # Use reference here
            "employmentStatus": random.choice(["employed", "self_employed", "unemployed", "student", "retired", "homemaker", "contractor"]),
            "monthlyIncomeUSD": random.randint(1000, 25000) if random.random() > 0.15 else None # Wider income range
        })
        if cust_info_ref["employmentStatus"] in ["employed", "self_employed", "contractor"]:
            cust_info_ref["occupation"] = fake.job()
            cust_info_ref["employer"] = fake.company() if cust_info_ref["employmentStatus"] == "employed" else "Self-Employed/Contractor"
    else: # Organization
        entity_doc.update({
            "incorporationDate": fake.date_of_birth(minimum_age=1, maximum_age=100).strftime("%Y-%m-%d"), # Wider range
            "jurisdictionOfIncorporation": addresses[0]["structured"]["country"]
        })
        cust_info_ref.update({ # Use reference here
            "industry": fake.bs(),
            "businessType": random.choice([
                "sole_proprietorship", "partnership", "llc", "s_corporation", "c_corporation",
                "non_profit_organization", "trust_entity", "holding_company", "joint_venture", "government_entity"
                ]),
            "numberOfEmployees": random.randint(1, 15000) if random.random() > 0.1 else None, # Wider range
            "annualRevenueUSD": random.randint(10000, 500000000) if random.random() > 0.1 else None # Wider range
        })
        # Add UBO Info
        entity_doc["uboInfo"] = kwargs.get("ubo_data", generate_ubo_data(existing_individual_pool=existing_individual_pool))


    # Allow kwargs to override any top-level or nested customerInfo fields
    for k, v in kwargs.items():
        if k == "customerInfo" and isinstance(v, dict):
            cust_info_ref.update(v) # Use reference here
        elif k in entity_doc and isinstance(entity_doc[k], dict) and isinstance(v, dict):
             entity_doc[k].update(v)
        elif k not in [ # list of kwargs handled by helper functions or specific logic above
            "first_name", "last_name", "company_name_base", "country_code", "city",
            "address_risk", "past_country_code_0", "past_country_code_1", "dob_str",
            "use_maiden_name_flag", "primary_address_struct", "primary_address_full",
            "primary_address_verified", "num_ids", "force_no_ids", "specific_ids",
            "primary_contact_type", "primary_contact_value", "source_system_override",
            "ubo_data", "existing_individual_pool", "force_addr_risk"
            ]:
            entity_doc[k] = v


    overall_score, level, components = calculate_detailed_risk(entity_doc)
    entity_doc["riskAssessment"] = {
        "overall": {"score": overall_score, "level": level, "trend": random.choice(["stable", "increasing", "decreasing"]),
                    "lastUpdated": updated_at,
                    "nextScheduledReview": updated_at + timedelta(days= (365 if level=="low" else (180 if level=="medium" else 60)) )}, # shorter review for high
        "components": components,
        "history": [{"date": updated_at, "score": overall_score, "level": level, "changeTrigger": "initial_assessment"}],
        "metadata": {"model": "aml_risk_v3.0", "assessmentType": "automated_initial", "overrides": []}
    }
    if entity_doc["resolution"]["status"] == "resolved" and not entity_doc["resolution"].get("masterEntityId"):
        entity_doc["resolution"]["masterEntityId"] = entity_doc["entityId"]

    # Enhanced Profile Summary Text
    summary_parts = []
    summary_parts.append(f"Entity Name: {entity_doc['name']['full']}.")
    if entity_doc['name'].get('aliases'): summary_parts.append(f"Aliases: {', '.join(entity_doc['name']['aliases'])}.")

    if entity_doc['entityType'] == 'individual':
        summary_parts.append(f"Type: Individual. DOB: {entity_doc.get('dateOfBirth', 'N/A')}. Gender: {entity_doc.get('gender', 'N/A')}.")
        summary_parts.append(f"Nationality: {', '.join(entity_doc.get('nationality',[]))}. Residency: {entity_doc.get('residency','N/A')}.")
        if cust_info_ref.get('occupation'): summary_parts.append(f"Occupation: {cust_info_ref['occupation']}.")
        if cust_info_ref.get('employer'): summary_parts.append(f"Employer: {cust_info_ref['employer']}.")
        if cust_info_ref.get('employmentStatus'): summary_parts.append(f"Employment: {cust_info_ref['employmentStatus']}.")
    else: # Organization
        summary_parts.append(f"Type: Organization. Incorporated: {entity_doc.get('incorporationDate', 'N/A')} in {entity_doc.get('jurisdictionOfIncorporation','N/A')}.")
        if cust_info_ref.get('industry'): summary_parts.append(f"Industry: {cust_info_ref['industry']}.")
        if cust_info_ref.get('businessType'): summary_parts.append(f"Business Type: {cust_info_ref['businessType']}.")
        if cust_info_ref.get('numberOfEmployees'): summary_parts.append(f"Employees: {cust_info_ref['numberOfEmployees']}.")
        if entity_doc.get("uboInfo"):
            summary_parts.append(f"UBOs Found: {len(entity_doc['uboInfo'])}.")
            for ubo_idx, ubo in enumerate(entity_doc["uboInfo"][:2]): # Summarize first 2 UBOs
                summary_parts.append(f" UBO {ubo_idx+1}: {ubo['name']} ({ubo['entityType']}, {ubo['percentageOwnership']}% ownership).")


    primary_address_found = False
    for addr_idx, addr in enumerate(entity_doc.get('addresses', [])):
        if addr.get('primary') and not addr.get('validTo'):
            primary_address_found = True
            summary_parts.append(f"Primary Address: {addr.get('full')}. Verified: {addr.get('verified')}.")
            break
        elif not primary_address_found and addr_idx == 0: # Fallback to first address if no primary current
             summary_parts.append(f"Main Address: {addr.get('full')}. Verified: {addr.get('verified')}.")

    for id_obj in entity_doc.get("identifiers", [])[:2]: # First 2 identifiers
        summary_parts.append(f"Identifier: {id_obj['type']} - {id_obj['value']} (Country: {id_obj['country']}, Verified: {id_obj['verified']}).")

    summary_parts.append(f"Risk Level: {entity_doc.get('riskAssessment',{}).get('overall',{}).get('level','N/A')}.")
    summary_parts.append(f"Risk Score: {entity_doc.get('riskAssessment',{}).get('overall',{}).get('score','N/A')}.")

    risk_components_calc = entity_doc.get('riskAssessment', {}).get('components', {}) # Use a different var name
    for comp_name, comp_data in risk_components_calc.items():
        if comp_data.get("factors"):
            summary_parts.append(f"Key {comp_name.capitalize()} Risk Factors ({comp_data['score']}):")
            for factor in comp_data["factors"][:2]: # Top 2 factors per component
                 summary_parts.append(f" - {factor.get('type', 'N/A')}: {factor.get('description', 'No description')[:100]}...")

    if entity_doc.get("watchlistMatches"):
        summary_parts.append("Watchlist Matches:")
        for match in entity_doc["watchlistMatches"][:2]:
            summary_parts.append(f" - List: {match.get('listId', 'N/A')}, Status: {match.get('status', 'N/A')}, Score: {match.get('matchScore', 'N/A')}.")

    entity_doc["profileSummaryText"] = " ".join(summary_parts)
    entity_doc["profileEmbedding"] = get_embedding_from_bedrock_or_fallback(entity_doc["profileSummaryText"], bedrock_runtime, EMBEDDING_MODEL_ID, EMBEDDING_DIMENSIONS)

    return entity_doc

print("Generating scenario-based entities with multiplication...")
all_entities_to_insert = []

# --- Scenario Multiplication Factors ---
NUM_CLEAR_DUPLICATE_SETS = 15
NUM_PEP_INDIVIDUALS = 20
NUM_EVOLVING_RISK_INDIVIDUALS = 10
NUM_SANCTIONED_ORGANIZATIONS = 8
NUM_COMPLEX_ORG_STRUCTURES = 5 # Each structure has multiple entities
NUM_INCOMPLETE_DATA_INDIVIDUALS = 25
NUM_HOUSEHOLD_SETS = 10 # Each set has 2 members
NUM_SUBTLE_DUPLICATE_CLUSTERS = 8 # Each cluster has 3 members
NUM_SHELL_COMPANY_CANDIDATES = 12
NUM_HNWI_INDIVIDUALS = 15

# --- SCENARIO 1: Clear Duplicate Pairs ---
print(f"Generating {NUM_CLEAR_DUPLICATE_SETS} sets of Clear Duplicates...")
for i in range(NUM_CLEAR_DUPLICATE_SETS):
    dup_dob_s1 = fake.date_of_birth(minimum_age=25, maximum_age=60).strftime("%Y-%m-%d")
    base_last_name = fake.last_name()
    s1_addr_struct_1 = {"street": fake.street_address(), "city": fake.city(), "state": fake.state_abbr(), "postalCode": fake.zipcode(), "country": "US"}
    s1_addr_full_1 = f"{s1_addr_struct_1['street']}, {s1_addr_struct_1['city']}, {s1_addr_struct_1['state']} {s1_addr_struct_1['postalCode']}, USA"

    ent_s1_dup1_id = generate_unique_id(f"CDI{i}A")
    ent_s1_dup1 = generate_entity_template("individual", scenario_key=f"clear_duplicate_set{i}_1", entityId=ent_s1_dup1_id,
                                     first_name=fake.first_name(), last_name=base_last_name, dob_str=dup_dob_s1,
                                     primary_address_struct=s1_addr_struct_1, primary_address_full=s1_addr_full_1,
                                     primary_contact_type="email", primary_contact_value=fake.email(),
                                     resolution={"status": "resolved", "masterEntityId": ent_s1_dup1_id})
    all_entities_to_insert.append(ent_s1_dup1); generated_entity_store[ent_s1_dup1_id] = ent_s1_dup1

    s1_addr_struct_2 = {**s1_addr_struct_1, "street": fake.street_address()} # Different street, same city/state
    s1_addr_full_2 = f"{s1_addr_struct_2['street']}, {s1_addr_struct_2['city']}, {s1_addr_struct_2['state']} {s1_addr_struct_2['postalCode']}, USA"
    ent_s1_dup2_id = generate_unique_id(f"CDI{i}B")
    ent_s1_dup2 = generate_entity_template("individual", scenario_key=f"clear_duplicate_set{i}_2", entityId=ent_s1_dup2_id,
                                     first_name=ent_s1_dup1["name"]["structured"]["first"][:3], last_name=base_last_name, dob_str=dup_dob_s1, # Nickname
                                     primary_address_struct=s1_addr_struct_2, primary_address_full=s1_addr_full_2,
                                     primary_contact_type="phone_mobile", primary_contact_value=fake.phone_number(),
                                     resolution={"status": "resolved", "masterEntityId": ent_s1_dup1_id, "confidence": random.uniform(0.85, 0.95),
                                                 "linkedEntities": [{"entityId": ent_s1_dup1_id, "linkType": "confirmed_match", "confidence": random.uniform(0.85,0.95), "matchedAttributes": ["dob", "last_name", "address_city_state_zip_fuzzy"], "matchDate": datetime.now(timezone.utc) - timedelta(days=random.randint(5,20)), "decidedBy": "analyst_synthetic_01", "decision": "confirmed_match"}],
                                                 "lastReviewDate": datetime.now(timezone.utc) - timedelta(days=random.randint(5,20)), "reviewedBy": "analyst_synthetic_01"})
    all_entities_to_insert.append(ent_s1_dup2); generated_entity_store[ent_s1_dup2_id] = ent_s1_dup2

# --- SCENARIO 2: PEP Individuals ---
print(f"Generating {NUM_PEP_INDIVIDUALS} PEP Individuals...")
for i in range(NUM_PEP_INDIVIDUALS):
    pep_id = generate_unique_id(f"PEP{i}")
    pep_country = random.choice(["US", "GB", "FR", "DE", "RU", "CN", "BR", "ZA"])
    pep_role = random.choice(["Senator", "Minister", "Ambassador", "Head of State Enterprise", "Senior Judge", "Military General"])
    pep_entity_kwargs = {
        "entityId":pep_id, "first_name":fake.first_name(), "last_name":fake.last_name() + " (PEP)", "dob_str":fake.date_of_birth(minimum_age=45, maximum_age=75).strftime("%Y-%m-%d"),
        "country_code":pep_country, "address_risk":"medium",
        "customerInfo":{"occupation": pep_role, "employmentStatus": "employed", "employer": f"{pep_country} Government Body"},
        "watchlistMatches":[
            {"listId": f"NATIONAL-PEP-{pep_country}", "matchId": f"PEP-{pep_id[:5]}", "matchScore": 0.99, "matchDate": datetime.now(timezone.utc) - timedelta(days=random.randint(10,100)), "status": "confirmed_hit", "details": {"role": pep_role, "country": pep_country, "source_reliability":"high"}}
        ],
        "status": random.choice(["active", "under_review"])
    }
    pep_entity = generate_entity_template("individual", scenario_key=f"pep_individual_varied_{i}", **pep_entity_kwargs)
    all_entities_to_insert.append(pep_entity); generated_entity_store[pep_id] = pep_entity
    potential_ubo_pool.append(pep_entity)

# --- SCENARIO 3: Evolving Risk Individuals ---
print(f"Generating {NUM_EVOLVING_RISK_INDIVIDUALS} Evolving Risk Individuals...")
for i in range(NUM_EVOLVING_RISK_INDIVIDUALS):
    evo_id = generate_unique_id(f"EVO{i}")
    evo_country = random.choice(["CA", "AU", "NZ", "US", "GB"])
    evolving_entity_kwargs = {
        "entityId":evo_id, "first_name":fake.first_name(), "last_name":fake.last_name(), "dob_str":fake.date_of_birth(minimum_age=22, maximum_age=40).strftime("%Y-%m-%d"),
        "country_code":evo_country, "address_risk":"low", "customerInfo": {"occupation":fake.job(), "employer":fake.company()}
    }
    evolving_entity = generate_entity_template("individual", scenario_key=f"evolving_risk_individual_{i}", **evolving_entity_kwargs)
    evolving_entity["riskAssessment"]["overall"].update({"score": random.randint(10,25), "level": "low"}) # Start low
    evolving_entity["riskAssessment"]["components"]["profile"]["score"] = random.randint(5,15)
    evolving_entity["riskAssessment"]["components"]["identity"]["score"] = random.randint(5,15)
    evolving_entity["riskAssessment"]["history"] = [{"date": evolving_entity["createdAt"], "score": evolving_entity["riskAssessment"]["overall"]["score"], "level": "low", "changeTrigger": "initial_assessment"}]
    all_entities_to_insert.append(evolving_entity); generated_entity_store[evo_id] = evolving_entity

# --- SCENARIO 4: Sanctioned Organizations ---
print(f"Generating {NUM_SANCTIONED_ORGANIZATIONS} Sanctioned Organizations...")
sanction_lists = {
    "OFAC-SDN-GEN": {"country": "US", "reason_keywords": ["terrorism", "proliferation", "narcotics"]},
    "EU-CONSOLIDATED-GEN": {"country": "EU", "reason_keywords": ["undermining sovereignty", "human rights abuses"]},
    "UN-SC-GEN": {"country": "UN", "reason_keywords": ["threat to peace", "arms embargo violation"]},
    "UK-HMT-GEN": {"country": "GB", "reason_keywords": ["financial sanctions", "terrorism financing"]}
}
for i in range(NUM_SANCTIONED_ORGANIZATIONS):
    sanctioned_org_id = generate_unique_id(f"SNO{i}")
    list_choice_key = random.choice(list(sanction_lists.keys()))
    list_details = sanction_lists[list_choice_key]
    org_country = random.choice(["IR", "SY", "KP", "VE", "RU", "MM"]) # High-risk countries

    sanctioned_org_kwargs = {
        "entityId":sanctioned_org_id, "company_name_base":fake.company_suffix().upper() + " " + fake.bs().capitalize().split(" ")[0] + " Global",
        "country_code":org_country, "address_risk":"high", "force_addr_risk":"high",
        "customerInfo":{"industry": random.choice(["Shipping", "Trading", "Manufacturing", "Financial Services"]), "businessType": random.choice(["c_corporation", "llc", "private_limited_company"])},
        "watchlistMatches":[
            {"listId": list_choice_key, "matchId": f"SANC-{org_country}-{i}", "matchScore": random.uniform(0.95, 1.0), "matchDate": datetime.now(timezone.utc) - timedelta(days=random.randint(5,365)), "status": "confirmed_hit", "details": {"reason": f"{random.choice(list_details['reason_keywords'])}, operating from {org_country}"}}
        ],
        "source_system_override": "regulatory_feed_processor",
        "status": random.choice(["inactive", "restricted", "under_review"])
    }
    sanctioned_org = generate_entity_template("organization", scenario_key=f"sanctioned_org_varied_{i}", **sanctioned_org_kwargs)
    all_entities_to_insert.append(sanctioned_org); generated_entity_store[sanctioned_org_id] = sanctioned_org

# --- SCENARIO 5: Complex Org Structures ---
print(f"Generating {NUM_COMPLEX_ORG_STRUCTURES} Complex Organization Structures...")
for i in range(NUM_COMPLEX_ORG_STRUCTURES):
    # Parent Company for structure 'i'
    parent_co_id = generate_unique_id(f"COPP{i}")
    parent_director_id = generate_unique_id(f"DIRP{i}")
    parent_co_director = generate_entity_template("individual", scenario_key=f"director_parent_struct{i}", entityId=parent_director_id,
                                               first_name=fake.first_name(), last_name=fake.last_name(), dob_str=fake.date_of_birth(minimum_age=40, maximum_age=70).strftime("%Y-%m-%d"),
                                               country_code=random.choice(["GB", "US", "LU", "CH"]), customerInfo={"occupation":"Executive Chairman"})
    all_entities_to_insert.append(parent_co_director); generated_entity_store[parent_director_id] = parent_co_director
    potential_ubo_pool.append(parent_co_director)

    parent_ubos = generate_ubo_data(num_ubos=1, existing_individual_pool=[parent_co_director])
    parent_ubos[0]["percentageOwnership"] = random.uniform(30,70)
    if random.random() < 0.5: # Add a corporate UBO or another individual UBO
        if random.random() < 0.6 and len(potential_ubo_pool) > 1:
            other_ubo_indiv = random.choice([p for p in potential_ubo_pool if p["entityId"] != parent_director_id])
            parent_ubos.append({
                "name": other_ubo_indiv["name"]["full"], "entityType":"individual", "nationality": other_ubo_indiv["nationality"][0] if other_ubo_indiv.get("nationality") else fake.country_code(),
                "percentageOwnership": random.uniform(10,40), "controlType": "indirect_ownership", "linkedEntityId": other_ubo_indiv["entityId"]
            })
        else:
             parent_ubos.append({
                "name": fake.company() + " Investments", "entityType": "corporate", "countryOfIncorporation": random.choice(["KY", "VG", "PA", "LU"]),
                "percentageOwnership": random.uniform(10,40), "controlType": "indirect_ownership_corporate", "linkedEntityId": None
            })

    parent_co = generate_entity_template("organization", scenario_key=f"complex_org_parent_struct{i}", entityId=parent_co_id,
                                     company_name_base=fake.word().capitalize() + " Group Holdings", country_code=parent_co_director["residency"], address_risk="low",
                                     customerInfo={"businessType":"holding_company", "industry":"Diversified Global Investments"}, ubo_data=parent_ubos)
    all_entities_to_insert.append(parent_co); generated_entity_store[parent_co_id] = parent_co

    # Subsidiaries for structure 'i' (1 to 3 subsidiaries)
    num_subsidiaries = random.randint(1,3)
    for j in range(num_subsidiaries):
        sub_co_id = generate_unique_id(f"COPS{i}_{j}")
        sub_director_id = generate_unique_id(f"DIRS{i}_{j}")
        sub_director = generate_entity_template("individual", scenario_key=f"director_sub_struct{i}_{j}", entityId=sub_director_id,
                                                   first_name=fake.first_name(), last_name=fake.last_name(), dob_str=fake.date_of_birth(minimum_age=35, maximum_age=60).strftime("%Y-%m-%d"),
                                                   country_code=random.choice(["DE", "FR", "SG", "HK", "AE"]), customerInfo={"occupation":"Managing Director"})
        all_entities_to_insert.append(sub_director); generated_entity_store[sub_director_id] = sub_director
        potential_ubo_pool.append(sub_director)

        sub_ubos = generate_ubo_data(num_ubos=random.randint(0,1), existing_individual_pool=[sub_director]) # Director might be UBO
        # Parent company is also a UBO (as corporate shareholder)
        sub_ubos.append({
            "name": parent_co["name"]["full"], "entityType": "corporate", "countryOfIncorporation": parent_co["jurisdictionOfIncorporation"],
            "percentageOwnership": random.uniform(51,100), "controlType": "direct_ownership_corporate_parent", "linkedEntityId": parent_co_id
        })

        sub_co = generate_entity_template("organization", scenario_key=f"complex_org_sub_struct{i}_{j}", entityId=sub_co_id,
                                         company_name_base=fake.bs().split(" ")[0] + " " + random.choice(["Tech", "Logistics", "Consulting", "Trading"]),
                                         country_code=sub_director["residency"], address_risk=random.choice(["low","medium"]),
                                         customerInfo={"industry":fake.bs()}, ubo_data=sub_ubos)
        all_entities_to_insert.append(sub_co); generated_entity_store[sub_co_id] = sub_co


# --- SCENARIO 6: Individual with Incomplete Data ---
print(f"Generating {NUM_INCOMPLETE_DATA_INDIVIDUALS} Incomplete Data Individuals...")
for i in range(NUM_INCOMPLETE_DATA_INDIVIDUALS):
    incomplete_id = generate_unique_id(f"INC{i}")
    inc_country = random.choice(["US", "GB", "CA", "AU", "??"]) # ?? for unknown country sometimes
    inc_kwargs = {
        "entityId":incomplete_id, "first_name":fake.first_name() if random.random() > 0.3 else "UnknownFN",
        "last_name":fake.last_name() if random.random() > 0.3 else "UnknownLN" + str(i),
        "dob_str":None if random.random() > 0.4 else fake.date_of_birth(minimum_age=18, maximum_age=80).strftime("%Y-%m-%d"),
        "primary_address_struct": {"street": fake.street_name(), "city": fake.city() if random.random() > 0.2 else "", "state": "", "postalCode": fake.postcode()[:3] if random.random() > 0.3 else "", "country": inc_country},
        "primary_address_verified": False,
        "force_no_ids": random.random() > 0.5,
        "num_ids": random.choices([0,1], weights=[0.6,0.4])[0], # More likely 0 or 1 ID
        "customerInfo": {"occupation": None if random.random() > 0.3 else fake.job(), "employmentStatus":random.choice(["unemployed", "self_employed", "other"])},
        "source_system_override": random.choice(["manual_entry_branch", "legacy_data_import_errors", "web_form_unverified"])
    }
    incomplete_entity = generate_entity_template("individual", scenario_key=f"incomplete_data_vague_{i}", **inc_kwargs)
    all_entities_to_insert.append(incomplete_entity); generated_entity_store[incomplete_id] = incomplete_entity

# --- SCENARIO 7: Household Sets ---
print(f"Generating {NUM_HOUSEHOLD_SETS} Household Sets...")
for i in range(NUM_HOUSEHOLD_SETS):
    household_addr_struct = {"street": fake.street_address(), "city": fake.city(), "state": fake.state_abbr(), "postalCode": fake.zipcode(), "country": "US"}
    household_addr_full = f"{household_addr_struct['street']}, {household_addr_struct['city']}, {household_addr_struct['state']} {household_addr_struct['postalCode']}, USA"
    common_last_name = fake.last_name()

    member1_id = generate_unique_id(f"CHMA{i}")
    member1 = generate_entity_template("individual", scenario_key=f"household_set{i}_member1", entityId=member1_id,
        first_name=fake.first_name_female(), last_name=common_last_name, dob_str=fake.date_of_birth(minimum_age=25, maximum_age=60).strftime("%Y-%m-%d"),
        primary_address_struct=household_addr_struct, primary_address_full=household_addr_full)
    all_entities_to_insert.append(member1); generated_entity_store[member1_id] = member1

    member2_id = generate_unique_id(f"CHMB{i}")
    member2 = generate_entity_template("individual", scenario_key=f"household_set{i}_member2", entityId=member2_id,
        first_name=fake.first_name_male(), last_name=common_last_name, dob_str=fake.date_of_birth(minimum_age=25, maximum_age=60).strftime("%Y-%m-%d"),
        primary_address_struct=household_addr_struct, primary_address_full=household_addr_full,
        use_maiden_name_flag= True if random.random() < 0.3 else False # One might have a maiden name alias
        )
    all_entities_to_insert.append(member2); generated_entity_store[member2_id] = member2
    potential_ubo_pool.extend([member1, member2])


# --- SCENARIO 8: Subtle Duplicate Clusters ---
print(f"Generating {NUM_SUBTLE_DUPLICATE_CLUSTERS} Subtle Duplicate Clusters...")
for i in range(NUM_SUBTLE_DUPLICATE_CLUSTERS):
    sdup_dob = fake.date_of_birth(minimum_age=28, maximum_age=55).strftime("%Y-%m-%d")
    sdup_last_name_base = fake.last_name()
    sdup_addr_base_street = fake.street_address()
    sdup_addr_city = fake.city()
    sdup_addr_country = random.choice(["US", "CA", "GB"])
    sdup_state = fake.state_abbr() if sdup_addr_country == "US" else fake.province_abbr() if sdup_addr_country == "CA" else ""
    sdup_zip = fake.zipcode() if sdup_addr_country == "US" else fake.postcode()
    sdup_phone = fake.phone_number()

    # Master for this cluster
    sdup_ent1_id = generate_unique_id(f"SDP{i}A")
    sdup_ent1_fn = fake.first_name()
    sdup_ent1 = generate_entity_template("individual", scenario_key=f"subtle_dup_cluster{i}_1_master", entityId=sdup_ent1_id,
                                     first_name=sdup_ent1_fn, last_name=sdup_last_name_base, dob_str=sdup_dob,
                                     primary_address_struct={"street": sdup_addr_base_street, "city": sdup_addr_city, "state":sdup_state, "postalCode":sdup_zip, "country":sdup_addr_country},
                                     primary_contact_type="email", primary_contact_value=fake.email(),
                                     resolution={"status": "resolved", "masterEntityId": sdup_ent1_id})
    all_entities_to_insert.append(sdup_ent1); generated_entity_store[sdup_ent1_id] = sdup_ent1

    # Duplicate 2
    sdup_ent2_id = generate_unique_id(f"SDP{i}B")
    sdup_ent2 = generate_entity_template("individual", scenario_key=f"subtle_dup_cluster{i}_2_variant", entityId=sdup_ent2_id,
                                     first_name=sdup_ent1_fn, middle_name=fake.first_name()[:1] if random.random() < 0.7 else "", last_name=sdup_last_name_base, dob_str=sdup_dob,
                                     primary_address_struct={"street": f"{sdup_addr_base_street}, Apt {random.randint(1,100)}{random.choice(['A','B','C',''])}", "city": sdup_addr_city, "state":sdup_state, "postalCode":sdup_zip, "country":sdup_addr_country},
                                     primary_contact_type="phone_mobile", primary_contact_value=sdup_phone, # Shared phone
                                     resolution={"status": "under_review", "masterEntityId": None, "confidence": random.uniform(0.6, 0.8),
                                                 "linkedEntities": [{"entityId": sdup_ent1_id, "linkType": "potential_duplicate", "confidence": random.uniform(0.6,0.8), "matchedAttributes": ["dob", "last_name", "address_fuzzy_street_city", "shared_phone_candidate"], "matchDate": datetime.now(timezone.utc) - timedelta(days=random.randint(1,10)), "decidedBy": "system_er_v2.1", "decision": "under_review"}]})
    all_entities_to_insert.append(sdup_ent2); generated_entity_store[sdup_ent2_id] = sdup_ent2

    # Duplicate 3
    sdup_ent3_id = generate_unique_id(f"SDP{i}C")
    dob_alt_format = datetime.strptime(sdup_dob, "%Y-%m-%d").strftime("%d/%m/%Y") if random.random() < 0.5 else sdup_dob # Mix DOB format

    # Define the addresses for sdup_ent3 separately for clarity
    sdup_ent3_addresses = [
        create_address_data(primary=True, country_code=sdup_addr_country, city=fake.city()), # A different current primary address
        create_address_data( # This is the past address linking to the cluster
            primary=False,
            address_type="previous", # This flag will ensure validTo is set in the past by create_address_data
            structured={
                "street": sdup_addr_base_street,
                "city": sdup_addr_city,
                "state":sdup_state,
                "postalCode":sdup_zip,
                "country":sdup_addr_country
            }
            # REMOVE validTo=datetime.now(timezone.utc)-timedelta(days=random.randint(200,500)) from here
        )
    ]
    # The create_address_data function when address_type="previous" or primary=False and it's not the first address
    # will internally set a validTo in the past. If you need more specific control for THIS EXACT past address's validTo,
    # you would generate it and then modify it:
    # sdup_ent3_addresses[1]["validTo"] = datetime.now(timezone.utc)-timedelta(days=random.randint(200,500)) # If specific control needed AFTER generation

    sdup_ent3 = generate_entity_template("individual", scenario_key=f"subtle_dup_cluster{i}_3_weaklinks", entityId=sdup_ent3_id,
                                     first_name=sdup_ent1_fn[0] + ".", last_name=sdup_last_name_base[:-1] + random.choice(['s','z','x']) if random.random() < 0.5 else sdup_last_name_base, # Initial, typo
                                     dob_str=dob_alt_format,
                                     addresses = sdup_ent3_addresses, # Assign the pre-generated list
                                     primary_contact_type="phone_mobile", primary_contact_value=sdup_phone, # Shared phone
                                     resolution={"status": "unresolved"})
    all_entities_to_insert.append(sdup_ent3); generated_entity_store[sdup_ent3_id] = sdup_ent3


# --- SCENARIO 9: Shell Company Candidates ---
print(f"Generating {NUM_SHELL_COMPANY_CANDIDATES} Shell Company Candidates...")
for i in range(NUM_SHELL_COMPANY_CANDIDATES):
    shell_co_id = generate_unique_id(f"SHL{i}")
    nominee_director_id = generate_unique_id(f"NOMD{i}")
    nom_country = random.choice(["VG", "KY", "PA", "SC", "BZ", "MT", "CY"]) # Typical nominee/offshore jurisdictions
    nominee_director = generate_entity_template("individual", scenario_key=f"nominee_director_shell{i}", entityId=nominee_director_id,
                                                first_name=random.choice(["Generic","Nominee","Service"]), last_name="Director " + str(random.randint(1000,9999)),
                                                dob_str=None, country_code=nom_country, address_risk="high", force_addr_risk="high",
                                                customerInfo={"occupation":"Corporate Services Provider"}, force_no_ids=True if random.random() < 0.7 else False)
    all_entities_to_insert.append(nominee_director); generated_entity_store[nominee_director_id] = nominee_director
    potential_ubo_pool.append(nominee_director)

    shell_ubos = generate_ubo_data(num_ubos=1, existing_individual_pool=[nominee_director])
    shell_ubos[0]["percentageOwnership"] = 100.0
    shell_ubos[0]["controlType"] = "nominee_shareholder_director"
    if random.random() < 0.2: # Sometimes, another layer of corporate UBO
        shell_ubos.append({
            "name": fake.company() + " Offshore Holdings Ltd.", "entityType": "corporate", "countryOfIncorporation": random.choice(["PA", "VG", "SC"]),
            "percentageOwnership": 100.0, "controlType": "corporate_beneficiary_unspecified_ultimate", "linkedEntityId": None
        })


    shell_co_country = random.choice(["KY", "VG", "PA", "AE", "HK", "SG", "MT", "CY"]) # Shells can be in various places
    shell_co = generate_entity_template("organization", scenario_key=f"shell_company_candidate_var{i}", entityId=shell_co_id,
                                    company_name_base=random.choice(["Alpha", "Beta", "Omega", "Prime", "Global", "Universal", "Strategic", "Dynamic"]) + " " + random.choice(["Consultants", "Trading", "Ventures", "Holdings", "Management", "Services"]) + " Ltd.",
                                    country_code=shell_co_country, address_risk="high", force_addr_risk="high",
                                    customerInfo={"industry": "General Business Activities / Holding", "businessType": "international_business_company", "numberOfEmployees": random.randint(0,3), "annualRevenueUSD": random.randint(0, 10000) if random.random() < 0.8 else None},
                                    ubo_data=shell_ubos, source_system_override="csp_onboarding_platform",
                                    status=random.choice(["active","dormant_pending_strike_off"]))
    shell_co["customerInfo"]["businessType"] = "shell_company_suspected"
    all_entities_to_insert.append(shell_co); generated_entity_store[shell_co_id] = shell_co

# --- SCENARIO 10: High Net Worth Individuals ---
print(f"Generating {NUM_HNWI_INDIVIDUALS} HNWIs...")
for i in range(NUM_HNWI_INDIVIDUALS):
    hnwi_id = generate_unique_id(f"HNWI{i}")
    primary_country = random.choice(["CH", "SG", "LU", "MC", "GB", "US"]) # Typical HNW residences
    hnwi_city = "Geneva" if primary_country == "CH" else "Singapore" if primary_country == "SG" else "Luxembourg City" if primary_country == "LU" else "Monaco" if primary_country == "MC" else fake.city()

    hnwi_primary_addr_struct = {"street": fake.street_address(), "city": hnwi_city, "state": "", "postalCode": fake.postcode(), "country": primary_country}

    hnwi_specific_ids_list = [create_identifier_data("individual", primary_country, id_type="passport")]
    if random.random() < 0.7: hnwi_specific_ids_list.append(create_identifier_data("individual", primary_country, id_type="national_id"))
    if random.random() < 0.5: hnwi_specific_ids_list.append(create_identifier_data("individual", random.choice(["US","GB", "CA"]), id_type="tax_id")) # Foreign tax id

    hnwi_instance = generate_entity_template("individual", scenario_key=f"hnwi_global_investor_{i}", entityId=hnwi_id,
                                    first_name=fake.first_name_nonbinary(), last_name=fake.last_name(), dob_str=fake.date_of_birth(minimum_age=40, maximum_age=75).strftime("%Y-%m-%d"),
                                    primary_address_struct=hnwi_primary_addr_struct, address_risk="low",
                                    customerInfo={
                                        "occupation":random.choice(["Private Equity Investor", "Real Estate Developer", "Retired Entrepreneur", "Art Collector", "Hedge Fund Manager"]), "employmentStatus":"self_employed",
                                        "segments":["private_wealth_management", "global_family_office_services"],
                                        "products":random.sample(["managed_investment_portfolio_bespoke", "offshore_trust_complex", "jumbo_mortgage_luxury_property", "art_secured_loan", "private_jet_financing", "philanthropic_advisory_services"], k=random.randint(2,4)),
                                        "monthlyIncomeUSD": random.randint(75000, 750000)
                                        },
                                    specific_ids = hnwi_specific_ids_list
                                    )
    # Add secondary/tertiary addresses in different (potentially medium/high risk) jurisdictions
    num_other_hnw_addrs = random.randint(1,2)
    for k in range(num_other_hnw_addrs):
        other_addr_country = random.choice(["KY", "AE", "MT", "CY", "PA", "BS", "VG"] if random.random() < 0.4 else ["GB", "FR", "US", "DE"]) # Mix of offshore and major financial centers
        other_addr_risk = "high" if other_addr_country in ["KY", "AE", "MT", "CY", "PA", "BS", "VG"] else "medium"
        hnwi_instance["addresses"].append(create_address_data(primary=False, country_code=other_addr_country, risk_level=other_addr_risk, address_type=random.choice(["investment_property", "vacation_home_address", "business_correspondence_offshore"])))

    all_entities_to_insert.append(hnwi_instance); generated_entity_store[hnwi_id] = hnwi_instance
    potential_ubo_pool.append(hnwi_instance)


# --- INCREASE GENERIC ENTITIES ---
NUM_GENERIC_INDIVIDUALS = 200 # Further increased
NUM_GENERIC_ORGANIZATIONS = 100  # Further increased

print(f"Generating {NUM_GENERIC_INDIVIDUALS} generic individuals...")
for i in range(NUM_GENERIC_INDIVIDUALS):
    ent_id = generate_unique_id(f"CGI{i}")
    entity = generate_entity_template("individual", scenario_key="generic_individual", entityId=ent_id,
                                      existing_individual_pool=potential_ubo_pool if i < NUM_GENERIC_INDIVIDUALS * 0.05 else None) # Smaller % links to existing UBO pool
    all_entities_to_insert.append(entity); generated_entity_store[ent_id] = entity
    if random.random() < 0.15: # Add 15% of generic individuals to UBO pool for future orgs
        potential_ubo_pool.append(entity)

print(f"Generating {NUM_GENERIC_ORGANIZATIONS} generic organizations...")
for i in range(NUM_GENERIC_ORGANIZATIONS):
    ent_id = generate_unique_id(f"CGO{i}")
    entity = generate_entity_template("organization", scenario_key="generic_organization", entityId=ent_id,
                                      existing_individual_pool=potential_ubo_pool if len(potential_ubo_pool)>0 else None)
    all_entities_to_insert.append(entity); generated_entity_store[ent_id] = entity

# --- Insert all entities ---
if all_entities_to_insert:
    print(f"\nTotal entities to insert: {len(all_entities_to_insert)}")
    print(f"Attempting to insert entities into '{entities_collection.name}' collection...")
    try:
        entities_collection.drop()
        result = entities_collection.insert_many(all_entities_to_insert, ordered=False)
        print(f"Successfully inserted {len(result.inserted_ids)} entities.")
    except pymongo.errors.BulkWriteError as bwe:
        print("Bulk write error during entity insertion:")
        for error_detail in bwe.details.get('writeErrors', []): print(f"  Index: {error_detail['index']}, Code: {error_detail['code']}, Message: {error_detail['errmsg']}")
    except Exception as e: print(f"An error occurred during entity insertion: {e}")
else: print("No entities were generated to insert.")

print("\n--- Entity Seeding with Scenario Multiplication Complete ---")

Generating scenario-based entities with multiplication...
Generating 15 sets of Clear Duplicates...
Generating 20 PEP Individuals...
Generating 10 Evolving Risk Individuals...
Generating 8 Sanctioned Organizations...
Generating 5 Complex Organization Structures...
Generating 25 Incomplete Data Individuals...
Generating 10 Household Sets...
Generating 8 Subtle Duplicate Clusters...
Generating 12 Shell Company Candidates...
Generating 15 HNWIs...
Generating 200 generic individuals...
Generating 100 generic organizations...

Total entities to insert: 504
Attempting to insert entities into 'entities' collection...
Successfully inserted 504 entities.

--- Entity Seeding with Scenario Multiplication Complete ---


# 2. `relationships` Collection

*   **Schema:** (Your schema is good)
    *   `_id`: `ObjectId()`
    *   `relationshipId`: String (e.g., "REL" + random) - **Unique**
    *   `source`: { `entityId`, `entityType` }
    *   `target`: { `entityId`, `entityType` }
    *   `type`: String (e.g., "potential_duplicate", "confirmed_same_entity", "household_member", "business_associate", "director_of", "shareholder_of", "transactional_link")
    *   `subType`: String (optional)
    *   `direction`: "bidirectional", "directed" (source->target)
    *   `strength`: Number (0-1, especially for ER or inferred links)
    *   `active`: Boolean
    *   `verified`, `verifiedBy`, `verificationDate`
    *   `evidence`: Array of Objects [{ `type`, `attribute`, `similarity`, `details` }]
    *   `created`, `updated`, `validFrom`, `validTo`: `ISODate()`
    *   `riskContribution`, `riskDirection`, `riskFactors`
    *   `datasource`: String (e.g., "entity_resolution", "manual_investigation", "transaction_analysis")
    *   `confidence`: Number
    *   `notes`, `tags`
    *   `createdBy`, `reviewStatus`, `reviewDate`, `reviewedBy`
*   **Data Tweaks for `relationships`:**
    1.  Populate based on the ER "duplicate" scenarios defined for entities.
        *   If Entity A and B are duplicates, create a `confirmed_same_entity` relationship.
    2.  Create explicit relationships:
        *   Entity H (`Peter Jones`) `director_of` Org G.
        *   Entity H (`Peter Jones`) `shareholder_of` Org I.
        *   Entity X and Y `household_member` (share same primary residential address).
    3.  Some relationships should be `active: false` or have `validTo` in the past to show temporal graph changes.

NEW RELATIONSHIPS

In [ ]:
relationships_collection = db["relationships"]
entities_collection = db["entities"] # Assumed to be populated

# --- Helper: Get Entity Reference for Relationships ---
def get_entity_ref_details(entity_id_val):
    """Fetches entityId, entityType, and full name for context."""
    if not entity_id_val: return None
    entity_doc = entities_collection.find_one(
        {"entityId": entity_id_val},
        {"entityId": 1, "entityType": 1, "name.full": 1, "_id": 0}
    )
    if entity_doc:
        return {"entityId": entity_doc["entityId"], "entityType": entity_doc["entityType"], "name": entity_doc.get("name", {}).get("full")}
    # print(f"WARN: Entity with entityId '{entity_id_val}' not found for relationship.")
    return None

def get_entity_ref_by_scenario(scenario_key_val):
    """Fetches entityId and entityType by scenarioKey."""
    if not scenario_key_val: return None
    entity_doc = entities_collection.find_one(
        {"scenarioKey": scenario_key_val},
        {"entityId": 1, "entityType": 1, "_id": 0}
    )
    if entity_doc:
        return {"entityId": entity_doc["entityId"], "entityType": entity_doc["entityType"]}
    # print(f"WARN: Entity with scenarioKey '{scenario_key_val}' not found for relationship ref.")
    return None


# --- Relationship Template ---
def generate_relationship_template(**kwargs):
    created_at = kwargs.get("created_at", fake.date_time_between(start_date="-4y", end_date="-1d", tzinfo=timezone.utc))
    updated_at = fake.date_time_between(start_date=created_at, end_date="now", tzinfo=timezone.utc)

    source_ref = kwargs.get("source_ref")
    target_ref = kwargs.get("target_ref")

    if not source_ref or not target_ref or not source_ref.get("entityId") or not target_ref.get("entityId"):
        # print(f"ERROR: Source or Target ref is None or missing entityId. Cannot create relationship. Source: {source_ref}, Target: {target_ref}")
        return None

    rel_type = kwargs.get("type")
    default_strength = 1.0
    if rel_type in ["potential_duplicate", "inferred_transactional_link", "social_media_connection_public", "known_associate_unverified", "business_associate_suspected"]:
        default_strength = round(random.uniform(0.3, 0.75), 2)
    elif rel_type in ["confirmed_same_entity", "shareholder_of", "director_of", "ubo_of", "parent_of_subsidiary", "household_member"]:
        default_strength = round(random.uniform(0.9, 1.0), 2)

    default_confidence = default_strength
    if rel_type == "potential_duplicate":
        default_confidence = round(random.uniform(0.4, 0.8), 2)

    rel = {
        "_id": ObjectId(),
        "relationshipId": f"REL-{uuid.uuid4().hex[:10].upper()}",
        "source": {"entityId": source_ref["entityId"], "entityType": source_ref["entityType"]},
        "target": {"entityId": target_ref["entityId"], "entityType": target_ref["entityType"]},
        "type": rel_type,
        "subType": kwargs.get("sub_type"),
        "direction": kwargs.get("direction", "bidirectional" if rel_type not in ["director_of", "ubo_of", "parent_of_subsidiary", "potential_beneficial_owner_of"] else "directed"),
        "strength": kwargs.get("strength", default_strength),
        "active": kwargs.get("active", True),
        "verified": kwargs.get("verified", rel_type not in ["potential_duplicate", "inferred_transactional_link", "social_media_connection_public", "known_associate_unverified", "business_associate_suspected"]),
        "verifiedBy": kwargs.get("verified_by"),
        "verificationDate": kwargs.get("verification_date"),
        "evidence": kwargs.get("evidence", []),
        "created": created_at,
        "updated": updated_at,
        "validFrom": kwargs.get("valid_from", created_at - timedelta(days=random.randint(0, 365*3))),
        "validTo": kwargs.get("valid_to"),
        "riskContribution": kwargs.get("risk_contribution", round(random.uniform(0.05, 0.6), 2) if kwargs.get("active", True) else 0.0),
        "datasource": kwargs.get("datasource", "synthetic_aml_rules_engine_v3.1"),
        "confidence": kwargs.get("confidence", default_confidence),
        "notes": kwargs.get("notes"),
        "tags": kwargs.get("tags", []),
        "createdBy": kwargs.get("created_by", "system_data_gen_aml_v3"),
        "reviewStatus": kwargs.get("review_status"),
        "reviewDate": kwargs.get("review_date"),
        "reviewedBy": kwargs.get("reviewed_by")
    }

    if not rel.get("reviewStatus"):
        rel["reviewStatus"] = "confirmed_verified" if rel["verified"] else "pending_review"

    if rel["verified"] and not rel.get("verifiedBy"):
        rel["verifiedBy"] = "system_auto_verified_data_load"
        rel["verificationDate"] = rel["created"] + timedelta(days=random.randint(1,10))

    for key in ["subType", "verifiedBy", "verificationDate", "validTo", "notes", "tags", "reviewDate", "reviewedBy"]:
        if rel.get(key) is None:
            if key in rel: del rel[key]

    if rel.get("validTo") and rel.get("validFrom"):
        if rel["validTo"] <= rel["validFrom"]:
            rel["validTo"] = rel["validFrom"] + timedelta(days=random.randint(30, 730))
        if not rel["active"] and rel.get("validTo") and rel["validTo"] > datetime.now(timezone.utc):
            rel["validTo"] = datetime.now(timezone.utc) - timedelta(days=random.randint(1,30))
            if rel["validFrom"] >= rel["validTo"]:
                 rel["validFrom"] = rel["validTo"] - timedelta(days=random.randint(30,100))
    elif not rel["active"] and not rel.get("validTo"):
        rel["validTo"] = rel.get("updated", rel["created"]) - timedelta(days=random.randint(1, 365))
        if rel.get("validFrom") and rel["validFrom"] >= rel["validTo"]:
            rel["validFrom"] = rel["validTo"] - timedelta(days=random.randint(30,100))
    return rel

# --- Define NUM_ constants (MUST MATCH YOUR ENTITY GENERATION SCRIPT) ---
NUM_CLEAR_DUPLICATE_SETS = 15
NUM_PEP_INDIVIDUALS = 20
NUM_EVOLVING_RISK_INDIVIDUALS = 10
NUM_SANCTIONED_ORGANIZATIONS = 8
NUM_COMPLEX_ORG_STRUCTURES = 5
# NUM_INCOMPLETE_DATA_INDIVIDUALS = 25 # Not directly used
NUM_HOUSEHOLD_SETS = 10
NUM_SUBTLE_DUPLICATE_CLUSTERS = 8
NUM_SHELL_COMPANY_CANDIDATES = 12
NUM_HNWI_INDIVIDUALS = 15
# --- End of NUM_ constants ---

# Enhanced Relationships Collection Generation
# This replaces the existing relationships generation code starting from "all_relationships_to_insert = []"
# Creates 500-600 relationships with multi-hop patterns (up to 4 degrees)
# Excludes geographic proximity and transaction-based relationships as requested

all_relationships_to_insert = []
print("Generating enhanced relationships with multi-hop patterns...")

# Store relationships by entity for multi-hop generation
entity_relationships_map = {}

def add_relationship_to_map(rel):
    """Helper to track relationships for multi-hop generation"""
    if rel and rel.get("source") and rel.get("target"):
        source_id = rel["source"]["entityId"]
        target_id = rel["target"]["entityId"]

        if source_id not in entity_relationships_map:
            entity_relationships_map[source_id] = {"outgoing": [], "incoming": []}
        if target_id not in entity_relationships_map:
            entity_relationships_map[target_id] = {"outgoing": [], "incoming": []}

        entity_relationships_map[source_id]["outgoing"].append(target_id)
        entity_relationships_map[target_id]["incoming"].append(source_id)

# --- 1. Entity Resolution Links (Duplicates) - Enhanced ---
print(f"Creating ER links for {NUM_CLEAR_DUPLICATE_SETS} Clear Duplicate sets...")
for i in range(NUM_CLEAR_DUPLICATE_SETS):
    master_ref = get_entity_ref_by_scenario(f"clear_duplicate_set{i}_1")
    dup_ref = get_entity_ref_by_scenario(f"clear_duplicate_set{i}_2")

    if master_ref and dup_ref:
        dup_entity_doc = entities_collection.find_one({"entityId": dup_ref["entityId"]}, {"resolution": 1})
        er_confidence = dup_entity_doc.get("resolution", {}).get("confidence", 0.92) if dup_entity_doc else 0.92
        rel_er = generate_relationship_template(
            source_ref=master_ref, target_ref=dup_ref, type="confirmed_same_entity",
            direction="bidirectional", strength=1.0, confidence=er_confidence, verified=True,
            verified_by="analyst_synthetic_01", verification_date=datetime.now(timezone.utc) - timedelta(days=random.randint(5,20)),
            evidence=[
                {"type": "attribute_match", "attribute": "dob", "similarity": 1.0, "details": "Exact DOB match"},
                {"type": "attribute_match", "attribute": "last_name", "similarity": 1.0, "details": "Exact last name match"},
                {"type": "attribute_match", "attribute": "address_fuzzy", "similarity": round(random.uniform(0.8,0.95),2), "details": "Highly similar address components"}
            ],
            datasource="entity_resolution_engine_v3", notes=f"Resolved as duplicate for set {i}."
        )
        if rel_er:
            all_relationships_to_insert.append(rel_er)
            add_relationship_to_map(rel_er)

print(f"Creating ER links for {NUM_SUBTLE_DUPLICATE_CLUSTERS} Subtle Duplicate Clusters...")
for i in range(NUM_SUBTLE_DUPLICATE_CLUSTERS):
    master_ref = get_entity_ref_by_scenario(f"subtle_dup_cluster{i}_1_master")
    variant2_ref = get_entity_ref_by_scenario(f"subtle_dup_cluster{i}_2_variant")
    variant3_ref = get_entity_ref_by_scenario(f"subtle_dup_cluster{i}_3_weaklinks")

    if master_ref and variant2_ref:
        var2_entity_doc = entities_collection.find_one({"entityId": variant2_ref["entityId"]}, {"resolution": 1})
        er_conf_v2 = var2_entity_doc.get("resolution", {}).get("confidence", 0.75) if var2_entity_doc else 0.75
        er_status_v2 = var2_entity_doc.get("resolution", {}).get("status", "under_review") if var2_entity_doc else "under_review"
        rel_er_v2 = generate_relationship_template(
            source_ref=master_ref, target_ref=variant2_ref, type="potential_duplicate",
            strength=er_conf_v2, confidence=er_conf_v2, verified= (er_status_v2 == "resolved"),
            review_status= "confirmed_match" if er_status_v2 == "resolved" else "pending_further_review",
            evidence=[{"type":"multiple_attribute_weak_match", "details":"Matches on DOB, partial name, fuzzy address, potential shared phone."}],
            datasource="entity_resolution_fuzzy_matcher_v2", notes=f"Potential duplicate in cluster {i} (variant 2)."
        )
        if rel_er_v2:
            all_relationships_to_insert.append(rel_er_v2)
            add_relationship_to_map(rel_er_v2)

    if master_ref and variant3_ref:
         rel_er_v3 = generate_relationship_template(
            source_ref=master_ref, target_ref=variant3_ref, type="potential_duplicate",
            strength=round(random.uniform(0.5,0.7),2), confidence=round(random.uniform(0.5,0.7),2), verified=False,
            review_status= "requires_manual_investigation",
            evidence=[{"type":"shared_phone_past_address", "details":"Shared phone number, and past address matches master's current."}],
            datasource="entity_resolution_linker_v1", notes=f"Weak potential duplicate in cluster {i} (variant 3)."
        )
         if rel_er_v3:
            all_relationships_to_insert.append(rel_er_v3)
            add_relationship_to_map(rel_er_v3)

# --- 2. Enhanced Organizational Structure & UBO Links with Multi-Level ---
print(f"Creating Enhanced Organizational Structure & UBO links for {NUM_COMPLEX_ORG_STRUCTURES} structures...")
for i in range(NUM_COMPLEX_ORG_STRUCTURES):
    parent_co_ref = get_entity_ref_by_scenario(f"complex_org_parent_struct{i}")
    parent_director_ref = get_entity_ref_by_scenario(f"director_parent_struct{i}")

    if not parent_co_ref:
        continue

    if parent_director_ref:
        rel_dir_parent = generate_relationship_template(
            source_ref=parent_director_ref, target_ref=parent_co_ref, type="director_of", direction="directed",
            sub_type="Executive Chairman/CEO", strength=1.0, verified=True,
            evidence=[{"type":"company_registry_simulated", "doc_ref":f"REG-P{i}-DIR"}]
        )
        if rel_dir_parent:
            all_relationships_to_insert.append(rel_dir_parent)
            add_relationship_to_map(rel_dir_parent)

    parent_co_doc = entities_collection.find_one({"entityId": parent_co_ref["entityId"]}, {"uboInfo":1, "name":1})
    if parent_co_doc and parent_co_doc.get("uboInfo"):
        for ubo in parent_co_doc["uboInfo"]:
            ubo_source_entity_ref = None
            if ubo.get("linkedEntityId"):
                ubo_source_entity_ref = get_entity_ref_details(ubo["linkedEntityId"])

            if ubo_source_entity_ref:
                rel_ubo_parent = generate_relationship_template(
                    source_ref=ubo_source_entity_ref, target_ref=parent_co_ref, type="ubo_of", direction="directed",
                    sub_type=f"{ubo.get('percentageOwnership', '')}% ownership via {ubo.get('controlType','N/A')}", strength=1.0, verified=True,
                    evidence=[{"type":"ubo_registry_declaration_simulated", "doc_ref":f"UBO-P{i}-{ubo_source_entity_ref['entityId'][:4]}"}],
                    notes=f"UBO {ubo.get('name', ubo_source_entity_ref.get('name','N/A'))} for {parent_co_ref.get('name', parent_co_ref['entityId'])}"
                )
                if rel_ubo_parent:
                    all_relationships_to_insert.append(rel_ubo_parent)
                    add_relationship_to_map(rel_ubo_parent)

    # Create subsidiaries and their relationships
    for j in range(3):
        sub_co_ref = get_entity_ref_by_scenario(f"complex_org_sub_struct{i}_{j}")
        if not sub_co_ref: continue

        sub_director_ref = get_entity_ref_by_scenario(f"director_sub_struct{i}_{j}")

        rel_parent_sub = generate_relationship_template(
            source_ref=parent_co_ref, target_ref=sub_co_ref, type="parent_of_subsidiary", direction="directed",
            sub_type="Majority Owned", strength=1.0, verified=True,
            evidence=[{"type":"group_structure_filing_sim", "doc_ref":f"GRP-STRUCT-{i}-{j}"}]
        )
        if rel_parent_sub:
            all_relationships_to_insert.append(rel_parent_sub)
            add_relationship_to_map(rel_parent_sub)

        if sub_director_ref:
            rel_dir_sub = generate_relationship_template(
                source_ref=sub_director_ref, target_ref=sub_co_ref, type="director_of", direction="directed",
                sub_type="Managing Director", strength=1.0, verified=True,
                evidence=[{"type":"company_registry_simulated", "doc_ref":f"REG-S{i}{j}-DIR"}]
            )
            if rel_dir_sub:
                all_relationships_to_insert.append(rel_dir_sub)
                add_relationship_to_map(rel_dir_sub)

# --- 3. Enhanced Business Relationships Between Organizations ---
print("Creating enhanced business relationships between organizations...")
all_orgs = list(entities_collection.find(
    {"entityType": "organization"},
    {"entityId":1, "entityType":1, "name.full":1, "customerInfo.industry":1, "jurisdictionOfIncorporation":1}
).limit(100))

if len(all_orgs) >= 20:
    # Create supplier-customer relationships
    for _ in range(40):  # Increased from 30
        if len(all_orgs) < 2: break
        supplier, customer = random.sample(all_orgs, 2)

        rel_business = generate_relationship_template(
            source_ref={"entityId": supplier["entityId"], "entityType": "organization", "name": supplier.get("name",{}).get("full")},
            target_ref={"entityId": customer["entityId"], "entityType": "organization", "name": customer.get("name",{}).get("full")},
            type="supplier_of", direction="directed",
            sub_type=random.choice(["Primary Supplier", "Secondary Supplier", "Exclusive Supplier"]),
            strength=round(random.uniform(0.6,0.9),2), verified=True,
            evidence=[{"type":"contract_database", "doc_ref":f"CONTRACT-{uuid.uuid4().hex[:8]}"}],
            datasource="procurement_system_integration"
        )
        if rel_business:
            all_relationships_to_insert.append(rel_business)
            add_relationship_to_map(rel_business)

    # Create joint venture relationships
    for _ in range(40):  # Increased from 25  # Increased from 15
        if len(all_orgs) < 2: break
        partner1, partner2 = random.sample(all_orgs, 2)

        rel_jv = generate_relationship_template(
            source_ref={"entityId": partner1["entityId"], "entityType": "organization", "name": partner1.get("name",{}).get("full")},
            target_ref={"entityId": partner2["entityId"], "entityType": "organization", "name": partner2.get("name",{}).get("full")},
            type="joint_venture_partner", direction="bidirectional",
            sub_type=f"{random.randint(30,70)}%-{100-random.randint(30,70)}% Partnership",
            strength=round(random.uniform(0.7,0.95),2), verified=True,
            evidence=[{"type":"jv_agreement", "doc_ref":f"JV-{uuid.uuid4().hex[:8]}"}],
            datasource="corporate_filings_database"
        )
        if rel_jv:
            all_relationships_to_insert.append(rel_jv)
            add_relationship_to_map(rel_jv)

# --- 4. Professional Service Provider Networks ---
print("Creating professional service provider networks...")
# Get some HNWIs and organizations to link to service providers
hnwi_list = list(entities_collection.find(
    {"scenarioKey": {"$regex": "^hnwi_global_investor_"}},
    {"entityId":1, "entityType":1, "name.full":1}
).limit(NUM_HNWI_INDIVIDUALS))

shell_companies = list(entities_collection.find(
    {"scenarioKey": {"$regex": "^shell_company_candidate_var"}},
    {"entityId":1, "entityType":1, "name.full":1}
).limit(NUM_SHELL_COMPANY_CANDIDATES))

# Create a few professional service providers (lawyers, accountants)
service_providers = []
for i in range(12):  # Increased from 8
    provider_type = random.choice(["law_firm", "accounting_firm", "wealth_management_firm", "trust_company"])
    provider_name = f"{fake.last_name()} & {fake.last_name()} {provider_type.replace('_', ' ').title()}"

    # Find or create a generic org to be the service provider
    provider_entity = entities_collection.find_one(
        {"entityType": "organization", "scenarioKey": "generic_organization"},
        {"entityId":1, "entityType":1, "name.full":1}
    )

    if provider_entity:
        service_providers.append({
            "entityId": provider_entity["entityId"],
            "entityType": "organization",
            "name": provider_entity.get("name",{}).get("full"),
            "service_type": provider_type
        })

# Link HNWIs and shell companies to service providers
for provider in service_providers[:6]:  # Use first 6 providers (increased from 4)
    # Each provider serves multiple clients
    num_clients = random.randint(5, 12)  # Increased from (3, 8)
    potential_clients = hnwi_list + shell_companies

    if len(potential_clients) >= num_clients:
        clients = random.sample(potential_clients, num_clients)

        for client in clients:
            service_type_map = {
                "law_firm": "legal_advisor_for",
                "accounting_firm": "accountant_for",
                "wealth_management_firm": "wealth_manager_for",
                "trust_company": "trustee_for"
            }

            rel_service = generate_relationship_template(
                source_ref=provider,
                target_ref={"entityId": client["entityId"], "entityType": client["entityType"], "name": client.get("name",{}).get("full")},
                type=service_type_map.get(provider["service_type"], "professional_service_provider"),
                direction="directed",
                strength=round(random.uniform(0.7,0.9),2),
                verified=random.random() > 0.3,
                evidence=[{"type":"client_onboarding_record", "doc_ref":f"CLIENT-{uuid.uuid4().hex[:8]}"}],
                datasource="professional_services_database",
                notes=f"{provider['name']} provides {provider['service_type'].replace('_', ' ')} services"
            )
            if rel_service:
                all_relationships_to_insert.append(rel_service)
                add_relationship_to_map(rel_service)

# --- 5. Multi-Hop Criminal/Suspicious Networks ---
print("Creating multi-hop suspicious networks...")
# Get sanctioned organizations and PEPs as network centers
sanctioned_orgs = list(entities_collection.find(
    {"scenarioKey": {"$regex": "^sanctioned_org_varied_"}},
    {"entityId":1, "entityType":1, "name.full":1}
).limit(NUM_SANCTIONED_ORGANIZATIONS))

peps = list(entities_collection.find(
    {"scenarioKey": {"$regex": "^pep_individual_varied_"}},
    {"entityId":1, "entityType":1, "name.full":1}
).limit(NUM_PEP_INDIVIDUALS))

# Create layered networks around high-risk entities
for center_entity in (sanctioned_orgs[:5] + peps[:5]):  # Use 5 of each as network centers (increased from 3)
    # First hop - direct associates
    num_direct_associates = random.randint(3, 6)  # Increased from (2, 4)
    potential_associates = list(entities_collection.find(
        {"entityType": {"$in": ["individual", "organization"]}, "entityId": {"$ne": center_entity["entityId"]}},
        {"entityId":1, "entityType":1, "name.full":1}
    ).limit(20))

    if len(potential_associates) >= num_direct_associates:
        direct_associates = random.sample(potential_associates, num_direct_associates)

        for associate in direct_associates:
            rel_type = random.choice([
                "business_associate_suspected",
                "financial_beneficiary_suspected",
                "proxy_relationship_suspected"
            ])

            rel_first_hop = generate_relationship_template(
                source_ref={"entityId": center_entity["entityId"], "entityType": center_entity["entityType"], "name": center_entity.get("name",{}).get("full")},
                target_ref={"entityId": associate["entityId"], "entityType": associate["entityType"], "name": associate.get("name",{}).get("full")},
                type=rel_type,
                direction="bidirectional",
                strength=round(random.uniform(0.5,0.8),2),
                verified=False,
                review_status="high_priority_investigation",
                evidence=[{"type":"transaction_pattern_analysis", "details":"Suspicious transaction patterns detected"}],
                datasource="network_analysis_engine",
                risk_contribution=round(random.uniform(0.3,0.7),2)
            )
            if rel_first_hop:
                all_relationships_to_insert.append(rel_first_hop)
                add_relationship_to_map(rel_first_hop)

            # Second hop - associates of associates
            if random.random() < 0.8:  # Increased from 0.6
                second_hop_candidates = list(entities_collection.find(
                    {"entityType": associate["entityType"],
                     "entityId": {"$nin": [center_entity["entityId"], associate["entityId"]]}},
                    {"entityId":1, "entityType":1, "name.full":1}
                ).limit(10))

                if second_hop_candidates:
                    second_hop_entity = random.choice(second_hop_candidates)

                    rel_second_hop = generate_relationship_template(
                        source_ref={"entityId": associate["entityId"], "entityType": associate["entityType"], "name": associate.get("name",{}).get("full")},
                        target_ref={"entityId": second_hop_entity["entityId"], "entityType": second_hop_entity["entityType"], "name": second_hop_entity.get("name",{}).get("full")},
                        type=random.choice(["known_associate_unverified", "business_partner", "financial_link_suspected"]),
                        direction="bidirectional",
                        strength=round(random.uniform(0.4,0.7),2),
                        verified=False,
                        datasource="extended_network_analysis",
                        notes="Second-degree connection in suspicious network"
                    )
                    if rel_second_hop:
                        all_relationships_to_insert.append(rel_second_hop)
                        add_relationship_to_map(rel_second_hop)

                    # Third hop - occasionally
                    if random.random() < 0.5:  # Increased from 0.3
                        third_hop_candidates = list(entities_collection.find(
                            {"entityId": {"$nin": [center_entity["entityId"], associate["entityId"], second_hop_entity["entityId"]]}},
                            {"entityId":1, "entityType":1, "name.full":1}
                        ).limit(5))

                        if third_hop_candidates:
                            third_hop_entity = random.choice(third_hop_candidates)

                            rel_third_hop = generate_relationship_template(
                                source_ref={"entityId": second_hop_entity["entityId"], "entityType": second_hop_entity["entityType"], "name": second_hop_entity.get("name",{}).get("full")},
                                target_ref={"entityId": third_hop_entity["entityId"], "entityType": third_hop_entity["entityType"], "name": third_hop_entity.get("name",{}).get("full")},
                                type="peripheral_connection",
                                direction="bidirectional",
                                strength=round(random.uniform(0.3,0.5),2),
                                verified=False,
                                datasource="deep_network_analysis",
                                notes="Third-degree connection in extended network"
                            )
                            if rel_third_hop:
                                all_relationships_to_insert.append(rel_third_hop)
                                add_relationship_to_map(rel_third_hop)

# --- 6. Family and Extended Social Networks ---
print("Creating extended family and social networks...")
# Extend household relationships to include extended family
household_members = list(entities_collection.find(
    {"scenarioKey": {"$regex": "^household_set.*_member"}},
    {"entityId":1, "entityType":1, "name.full":1, "scenarioKey":1}
))

# Create extended family relationships
for member in household_members[:20]:  # Process first 20 household members (increased from 10)
    # Add parents, siblings, in-laws
    extended_family_candidates = list(entities_collection.find(
        {"entityType": "individual", "entityId": {"$ne": member["entityId"]}},
        {"entityId":1, "entityType":1, "name.full":1}
    ).limit(15))

    if len(extended_family_candidates) >= 3:
        num_family = random.randint(1, 3)
        family_members = random.sample(extended_family_candidates, num_family)

        for family in family_members:
            rel_type = random.choice([
                "family_member_parent", "family_member_sibling",
                "family_member_child", "family_member_in_law"
            ])

            rel_family = generate_relationship_template(
                source_ref={"entityId": member["entityId"], "entityType": "individual", "name": member.get("name",{}).get("full")},
                target_ref={"entityId": family["entityId"], "entityType": "individual", "name": family.get("name",{}).get("full")},
                type=rel_type,
                direction="bidirectional",
                strength=round(random.uniform(0.8,0.95),2),
                verified=True,
                evidence=[{"type":"identity_document_relationship", "details":"Family relationship verified through documentation"}],
                datasource="kyc_family_verification"
            )
            if rel_family:
                all_relationships_to_insert.append(rel_family)
                add_relationship_to_map(rel_family)

# --- 7. Financial Institution Relationships ---
print("Creating financial institution relationships...")
# Get financial services organizations
fin_orgs = list(entities_collection.find(
    {"entityType": "organization", "$or": [
        {"customerInfo.industry": {"$regex": "financ|bank|invest", "$options": "i"}},
        {"name.full": {"$regex": "bank|financial|capital|invest", "$options": "i"}}
    ]},
    {"entityId":1, "entityType":1, "name.full":1}
).limit(20))

# Create lending relationships
individuals_needing_loans = list(entities_collection.find(
    {"entityType": "individual"},
    {"entityId":1, "entityType":1, "name.full":1}
).limit(50))

if fin_orgs and individuals_needing_loans:
    for _ in range(25):
        lender = random.choice(fin_orgs)
        borrower = random.choice(individuals_needing_loans)

        loan_type = random.choice([
            "mortgage_loan", "personal_loan", "business_loan",
            "line_of_credit", "auto_loan"
        ])

        rel_loan = generate_relationship_template(
            source_ref={"entityId": lender["entityId"], "entityType": "organization", "name": lender.get("name",{}).get("full")},
            target_ref={"entityId": borrower["entityId"], "entityType": borrower["entityType"], "name": borrower.get("name",{}).get("full")},
            type="loan_provider_for",
            sub_type=loan_type,
            direction="directed",
            strength=round(random.uniform(0.7,0.9),2),
            verified=True,
            evidence=[{"type":"loan_agreement", "doc_ref":f"LOAN-{uuid.uuid4().hex[:8]}"}],
            datasource="lending_platform",
            additional_details={"loan_amount": random.randint(10000, 1000000)}
        )
        if rel_loan:
            all_relationships_to_insert.append(rel_loan)
            add_relationship_to_map(rel_loan)

# --- 8. Employment and Board Networks ---
print("Creating employment and board member networks...")
# Get organizations and individuals for employment relationships
orgs_for_employment = list(entities_collection.find(
    {"entityType": "organization"},
    {"entityId":1, "entityType":1, "name.full":1}
).limit(50))

individuals_for_employment = list(entities_collection.find(
    {"entityType": "individual", "customerInfo.employmentStatus": {"$in": ["employed", "self_employed"]}},
    {"entityId":1, "entityType":1, "name.full":1}
).limit(100))

if orgs_for_employment and individuals_for_employment:
    # Create employment relationships
    for _ in range(60):  # Increased from 40
        employer = random.choice(orgs_for_employment)
        employee = random.choice(individuals_for_employment)

        employment_type = random.choice([
            "full_time_employee", "contractor", "consultant",
            "part_time_employee", "advisor"
        ])

        rel_employment = generate_relationship_template(
            source_ref={"entityId": employee["entityId"], "entityType": "individual", "name": employee.get("name",{}).get("full")},
            target_ref={"entityId": employer["entityId"], "entityType": "organization", "name": employer.get("name",{}).get("full")},
            type="employed_by",
            sub_type=employment_type,
            direction="directed",
            strength=round(random.uniform(0.7,0.95),2),
            verified=True,
            evidence=[{"type":"employment_verification", "doc_ref":f"EMP-{uuid.uuid4().hex[:8]}"}],
            datasource="hr_system_integration"
        )
        if rel_employment:
            all_relationships_to_insert.append(rel_employment)
            add_relationship_to_map(rel_employment)

    # Create board member relationships (individuals serving on multiple boards)
    board_candidates = random.sample(individuals_for_employment, min(20, len(individuals_for_employment)))

    for board_member in board_candidates[:15]:  # Increased from 10
        # Each board member serves on 2-5 boards (increased from 2-4)
        num_boards = random.randint(2, 5)
        if len(orgs_for_employment) >= num_boards:
            boards = random.sample(orgs_for_employment, num_boards)

            for org in boards:
                rel_board = generate_relationship_template(
                    source_ref={"entityId": board_member["entityId"], "entityType": "individual", "name": board_member.get("name",{}).get("full")},
                    target_ref={"entityId": org["entityId"], "entityType": "organization", "name": org.get("name",{}).get("full")},
                    type="board_member_of",
                    sub_type=random.choice(["Independent Director", "Executive Director", "Advisory Board"]),
                    direction="directed",
                    strength=0.9,
                    verified=True,
                    evidence=[{"type":"board_appointment", "doc_ref":f"BOARD-{uuid.uuid4().hex[:8]}"}],
                    datasource="corporate_governance_db"
                )
                if rel_board:
                    all_relationships_to_insert.append(rel_board)
                    add_relationship_to_map(rel_board)

# --- Keep existing relationship types from original code ---
# (Include the household, high-risk entity links, past relationships, social links, and evolving risk relationships)
# [Original code sections would be inserted here]
# --- 3. Household Links ---
print(f"Creating Household links for {NUM_HOUSEHOLD_SETS} sets...")
for i in range(NUM_HOUSEHOLD_SETS):
    member1_ref = get_entity_ref_by_scenario(f"household_set{i}_member1")
    member2_ref = get_entity_ref_by_scenario(f"household_set{i}_member2")

    if member1_ref and member2_ref:
        rel_household = generate_relationship_template(
            source_ref=member1_ref, target_ref=member2_ref, type="household_member",
            direction="bidirectional", strength=0.95, verified=True,
            evidence=[{"type": "shared_primary_address_confirmed", "details": f"Shared address for household set {i}."}],
            datasource="synthetic_scenario_design_v2"
        )
        if rel_household: all_relationships_to_insert.append(rel_household)

# --- 4. Links involving High-Risk / Watchlisted Entities ---
print("Creating links involving High-Risk/Watchlisted entities...")
pep_sample = list(entities_collection.find({"scenarioKey": {"$regex": "^pep_individual_varied_"}}, {"entityId":1, "entityType":1, "name.full":1}).limit(NUM_PEP_INDIVIDUALS // 2 + 1))
hnwi_sample = list(entities_collection.find({"scenarioKey": {"$regex": "^hnwi_global_investor_"}}, {"entityId":1, "entityType":1, "name.full":1}).limit(NUM_HNWI_INDIVIDUALS // 2 + 1))
shell_co_sample = list(entities_collection.find({"scenarioKey": {"$regex": "^shell_company_candidate_var"}}, {"entityId":1, "entityType":1, "name.full":1}).limit(NUM_SHELL_COMPANY_CANDIDATES // 2 + 1))

for pep_entity_dict in pep_sample:
    pep_ref = {"entityId": pep_entity_dict["entityId"], "entityType": pep_entity_dict["entityType"], "name": pep_entity_dict.get("name",{}).get("full")}
    if random.random() < 0.4 and hnwi_sample:
        target_hnwi_dict = random.choice(hnwi_sample)
        target_ref = {"entityId": target_hnwi_dict["entityId"], "entityType": target_hnwi_dict["entityType"], "name": target_hnwi_dict.get("name",{}).get("full")}
        rel_pep_hnwi = generate_relationship_template(
            source_ref=pep_ref, target_ref=target_ref, type="business_associate_suspected", direction="bidirectional",
            strength=round(random.uniform(0.5,0.7),2), verified=False, review_status="requires_investigation",
            notes=f"Suspected association between PEP {pep_ref.get('name','N/A')} and HNWI {target_ref.get('name','N/A')}.",
            datasource="intelligence_leak_simulated_v2"
        )
        if rel_pep_hnwi: all_relationships_to_insert.append(rel_pep_hnwi)

    if random.random() < 0.3 and shell_co_sample:
        target_shell_dict = random.choice(shell_co_sample)
        target_ref = {"entityId": target_shell_dict["entityId"], "entityType": target_shell_dict["entityType"], "name": target_shell_dict.get("name",{}).get("full")}
        rel_pep_shell = generate_relationship_template(
            source_ref=pep_ref, target_ref=target_ref, type="potential_beneficial_owner_of", direction="directed",
            strength=round(random.uniform(0.4,0.65),2), verified=False, review_status="high_priority_investigation",
            notes=f"PEP {pep_ref.get('name','N/A')} potentially linked to shell company {target_ref.get('name','N/A')}.",
            datasource="financial_investigation_unit_tipoff_sim"
        )
        if rel_pep_shell: all_relationships_to_insert.append(rel_pep_shell)

sanctioned_org_sample = list(entities_collection.find({"scenarioKey": {"$regex": "^sanctioned_org_varied_"}}, {"entityId":1, "entityType":1, "name.full":1}).limit(NUM_SANCTIONED_ORGANIZATIONS // 2 + 1))
other_org_sample = list(entities_collection.find(
    {"entityType": "organization", "scenarioKey":{"$not": {"$regex":"^sanctioned_org_varied_"}}},
    {"entityId":1, "entityType":1, "name.full":1}
).limit(30))

if sanctioned_org_sample and other_org_sample:
    for sn_org_dict in sanctioned_org_sample:
        sn_org_ref = {"entityId": sn_org_dict["entityId"], "entityType": sn_org_dict["entityType"], "name": sn_org_dict.get("name",{}).get("full")}
        if random.random() < 0.6: # Link to 1-2 other orgs
            for _ in range(random.randint(1,2)):
                if not other_org_sample: break # Break if no more other orgs to pick from
                target_org_dict = random.choice(other_org_sample)
                target_org_ref = {"entityId": target_org_dict["entityId"], "entityType": target_org_dict["entityType"], "name": target_org_dict.get("name",{}).get("full")}
                if target_org_ref["entityId"] == sn_org_ref["entityId"]: continue

                rel_sn_other = generate_relationship_template(
                    source_ref=sn_org_ref, target_ref=target_org_ref, type="transactional_counterparty_high_risk", direction="bidirectional",
                    strength=round(random.uniform(0.6,0.8),2), verified=False,
                    notes=f"High-risk transactions detected between sanctioned org {sn_org_ref.get('name','N/A')} and {target_org_ref.get('name','N/A')}.",
                    datasource="transaction_monitoring_alert_sim_v2"
                )
                if rel_sn_other: all_relationships_to_insert.append(rel_sn_other)

# --- 5. Some Past/Inactive Relationships ---
print("Creating some Past/Inactive relationships...")
# Use the full potential_ubo_pool which contains various individuals
all_individuals_for_past_roles = list(entities_collection.find({"entityType":"individual"}, {"entityId":1, "entityType":1, "name.full":1}).limit(50))
all_orgs_for_past_roles = list(entities_collection.find({"entityType":"organization"}, {"entityId":1, "entityType":1, "name.full":1}).limit(50))

if len(all_individuals_for_past_roles) > 5 and len(all_orgs_for_past_roles) > 2 :
    for _ in range(max(5, NUM_COMPLEX_ORG_STRUCTURES)): # Create a few more past directorships
        director_dict = random.choice(all_individuals_for_past_roles)
        director_ref = {"entityId": director_dict["entityId"], "entityType": director_dict["entityType"], "name": director_dict.get("name",{}).get("full")}

        past_org_dict = random.choice(all_orgs_for_past_roles)
        past_org_ref = {"entityId": past_org_dict["entityId"], "entityType": past_org_dict["entityType"], "name": past_org_dict.get("name",{}).get("full")}

        if director_ref["entityId"] == past_org_ref["entityId"]: continue # Should not happen with type check but safety

        valid_from_past = fake.date_time_between(start_date="-10y", end_date="-3y", tzinfo=timezone.utc)
        valid_to_past = fake.date_time_between(start_date=valid_from_past + timedelta(days=365), end_date="-1y", tzinfo=timezone.utc)

        rel_past_directorship = generate_relationship_template(
            source_ref=director_ref, target_ref=past_org_ref,
            type="director_of", direction="directed", active=False, verified=True,
            validFrom=valid_from_past, validTo=valid_to_past,
            notes=f"Past directorship of {director_ref.get('name','N/A')} at {past_org_ref.get('name','N/A')}, resigned/term ended.",
            datasource="historical_corporate_filings_sim_v2"
        )
        if rel_past_directorship: all_relationships_to_insert.append(rel_past_directorship)

# --- 6. Generic "Social" or "Professional" Links for Network Density ---
print("Creating a few generic social/professional links for graph density...")
generic_individuals = list(entities_collection.find({"scenarioKey":"generic_individual"}, {"entityId":1, "entityType":1, "name.full":1}).limit(60))
if len(generic_individuals) >= 10:
    for _ in range(25): # Increased slightly
        if len(generic_individuals) < 2: break
        source_ind_dict, target_ind_dict = random.sample(generic_individuals, 2)
        source_ref = {"entityId": source_ind_dict["entityId"], "entityType": source_ind_dict["entityType"], "name": source_ind_dict.get("name",{}).get("full")}
        target_ref = {"entityId": target_ind_dict["entityId"], "entityType": target_ind_dict["entityType"], "name": target_ind_dict.get("name",{}).get("full")}

        rel_social = generate_relationship_template(
            source_ref=source_ref, target_ref=target_ref,
            type=random.choice(["professional_colleague_public", "social_media_connection_public", "known_associate_unverified"]),
            strength=round(random.uniform(0.3,0.6),2), verified=False,
            datasource="public_domain_data_scrape_sim_v2"
        )
        if rel_social: all_relationships_to_insert.append(rel_social)


# --- 7. Links for Evolving Risk Individuals (to showcase potential future risk) ---
print("Creating some speculative links for Evolving Risk Individuals...")
evolving_risk_sample = list(entities_collection.find(
    {"scenarioKey": {"$regex": "^evolving_risk_individual_"}},
    {"entityId":1, "entityType":1, "name.full":1}
).limit(max(5, NUM_EVOLVING_RISK_INDIVIDUALS // 2))) # NUM_EVOLVING_RISK_INDIVIDUALS from entity script

# Sample of other entities they might connect to:
# Could be other generic individuals, or even HNWIs, or less critical orgs
other_individuals_sample = list(entities_collection.find(
    {"entityType": "individual", "scenarioKey": {"$not": {"$regex": "^evolving_risk_individual_"}}}, # Not another evolving risk
    {"entityId":1, "entityType":1, "name.full":1}
).limit(30))

other_orgs_sample = list(entities_collection.find(
    {"entityType": "organization", "scenarioKey": {"$not": {"$regex": "^sanctioned_org_varied_"}}}, # Not sanctioned
    {"entityId":1, "entityType":1, "name.full":1}
).limit(20))


if evolving_risk_sample:
    for evo_entity_dict in evolving_risk_sample:
        evo_ref = {"entityId": evo_entity_dict["entityId"], "entityType": evo_entity_dict["entityType"], "name": evo_entity_dict.get("name",{}).get("full")}

        # Chance to link to another individual (e.g., new business partner, associate)
        if random.random() < 0.4 and other_individuals_sample:
            target_ind_dict = random.choice(other_individuals_sample)
            target_ref = {"entityId": target_ind_dict["entityId"], "entityType": target_ind_dict["entityType"], "name": target_ind_dict.get("name",{}).get("full")}
            if evo_ref["entityId"] == target_ref["entityId"]: continue

            rel_evo_ind = generate_relationship_template(
                source_ref=evo_ref, target_ref=target_ref,
                type="emerging_business_associate", # A custom type for this
                direction="bidirectional",
                strength=round(random.uniform(0.4, 0.6),2),
                verified=False,
                review_status="monitoring_activity",
                notes=f"New association noted between {evo_ref.get('name','N/A')} and {target_ref.get('name','N/A')}. Monitor for risk changes.",
                datasource="internal_observation_log_sim"
            )
            if rel_evo_ind: all_relationships_to_insert.append(rel_evo_ind)

        # Chance to link to an organization (e.g., becomes director of a small/new company, or consultant for)
        if random.random() < 0.3 and other_orgs_sample:
            target_org_dict = random.choice(other_orgs_sample)
            target_ref = {"entityId": target_org_dict["entityId"], "entityType": target_org_dict["entityType"], "name": target_org_dict.get("name",{}).get("full")}

            rel_type_org = random.choice(["consultant_for", "director_of_small_entity", "shareholder_minority_private_co"])
            rel_evo_org = generate_relationship_template(
                source_ref=evo_ref, target_ref=target_ref,
                type=rel_type_org,
                direction="directed",
                strength=round(random.uniform(0.5, 0.75),2),
                verified= (rel_type_org != "consultant_for"), # Directorship/shareholding might be verifiable
                notes=f"{evo_ref.get('name','N/A')} now linked to organization {target_ref.get('name','N/A')} as {rel_type_org}.",
                datasource="business_intelligence_update_sim"
            )
            if rel_evo_org: all_relationships_to_insert.append(rel_evo_org)

# --- 9. Create Fourth-Hop Relationships Using the Map ---
print("Creating fourth-hop relationships based on existing network...")
# Find entities that are already 3 hops away from high-risk entities
high_risk_entities = list(entities_collection.find(
    {"$or": [
        {"scenarioKey": {"$regex": "sanctioned"}},
        {"scenarioKey": {"$regex": "pep"}},
        {"riskAssessment.overall.level": "high"}
    ]},
    {"entityId":1}
).limit(5))

for high_risk_entity in high_risk_entities:
    entity_id = high_risk_entity["entityId"]

    # Find 3-hop connections
    if entity_id in entity_relationships_map:
        # First hop
        first_hops = entity_relationships_map[entity_id].get("outgoing", [])

        for first_hop_id in first_hops[:2]:  # Limit to avoid explosion
            if first_hop_id in entity_relationships_map:
                # Second hop
                second_hops = entity_relationships_map[first_hop_id].get("outgoing", [])

                for second_hop_id in second_hops[:2]:
                    if second_hop_id in entity_relationships_map:
                        # Third hop
                        third_hops = entity_relationships_map[second_hop_id].get("outgoing", [])

                        for third_hop_id in third_hops[:1]:
                            # Create fourth hop
                            fourth_hop_candidates = list(entities_collection.find(
                                {"entityId": {"$nin": [entity_id, first_hop_id, second_hop_id, third_hop_id]}},
                                {"entityId":1, "entityType":1, "name.full":1}
                            ).limit(3))

                            if fourth_hop_candidates:
                                fourth_hop = random.choice(fourth_hop_candidates)
                                third_hop_entity = entities_collection.find_one({"entityId": third_hop_id})

                                if third_hop_entity:
                                    rel_fourth_hop = generate_relationship_template(
                                        source_ref={"entityId": third_hop_id, "entityType": third_hop_entity["entityType"], "name": third_hop_entity.get("name",{}).get("full")},
                                        target_ref={"entityId": fourth_hop["entityId"], "entityType": fourth_hop["entityType"], "name": fourth_hop.get("name",{}).get("full")},
                                        type="distant_connection",
                                        sub_type="fourth_degree_separation",
                                        direction="bidirectional",
                                        strength=round(random.uniform(0.2,0.4),2),
                                        verified=False,
                                        datasource="extended_network_analysis",
                                        notes=f"Fourth-degree connection from high-risk entity",
                                        tags=["extended_network", "requires_monitoring"]
                                    )
                                    if rel_fourth_hop:
                                        all_relationships_to_insert.append(rel_fourth_hop)

# --- Clean up and insert relationships ---
if all_relationships_to_insert:
    unique_rels_dict = {}
    valid_relationships = []

    for rel_item in all_relationships_to_insert:
        if rel_item is None: continue

        s_id = rel_item["source"]["entityId"]
        t_id = rel_item["target"]["entityId"]

        # Create unique key
        if rel_item["direction"] == "directed":
            key = (s_id, t_id, rel_item["type"])
        else:
            key = (tuple(sorted((s_id, t_id))), rel_item["type"])

        if key not in unique_rels_dict:
            unique_rels_dict[key] = rel_item
            valid_relationships.append(rel_item)

    print(f"\nTotal unique relationships to insert: {len(valid_relationships)}")

    # Add more relationships if needed to reach 500-600
    if len(valid_relationships) < 500:
        print(f"Adding additional generic relationships to reach target...")

        # Add more generic business relationships
        generic_individuals = list(entities_collection.find(
            {"entityType": "individual"},
            {"entityId":1, "entityType":1, "name.full":1}
        ).limit(200))

        generic_orgs = list(entities_collection.find(
            {"entityType": "organization"},
            {"entityId":1, "entityType":1, "name.full":1}
        ).limit(100))

        while len(valid_relationships) < 600 and len(generic_individuals) >= 2:
            # Add various relationship types
            rel_type_options = [
                ("professional_colleague", "bidirectional", 0.5),
                ("former_associate", "bidirectional", 0.4),
                ("referral_source", "directed", 0.6),
                ("mentor_mentee", "directed", 0.7),
                ("alumni_connection", "bidirectional", 0.3),
                ("industry_contact", "bidirectional", 0.4),
                ("business_referral", "directed", 0.5),
                ("past_business_partner", "bidirectional", 0.4),
                ("investment_advisor", "directed", 0.6),
                ("legal_representative", "directed", 0.7)
            ]

            if random.random() < 0.7 and len(generic_individuals) >= 2:
                # Individual to individual
                source, target = random.sample(generic_individuals, 2)
                rel_type, direction, base_strength = random.choice(rel_type_options)
            else:
                # Individual to organization or org to org
                if generic_individuals and generic_orgs:
                    if random.random() < 0.5:
                        source = random.choice(generic_individuals)
                        target = random.choice(generic_orgs)
                        rel_type = random.choice(["customer_of", "vendor_for", "affiliated_with"])
                    else:
                        source = random.choice(generic_orgs)
                        target = random.choice(generic_orgs)
                        rel_type = random.choice(["competitor_of", "partner_with", "subsidiary_of"])
                    direction = "directed" if rel_type in ["customer_of", "vendor_for", "subsidiary_of"] else "bidirectional"
                    base_strength = 0.5
                else:
                    continue

            generic_rel = generate_relationship_template(
                source_ref={"entityId": source["entityId"], "entityType": source["entityType"], "name": source.get("name",{}).get("full")},
                target_ref={"entityId": target["entityId"], "entityType": target["entityType"], "name": target.get("name",{}).get("full")},
                type=rel_type,
                direction=direction,
                strength=round(random.uniform(base_strength-0.2, base_strength+0.2),2),
                verified=random.random() > 0.5,
                datasource="various_sources"
            )

            if generic_rel:
                # Check uniqueness
                s_id = generic_rel["source"]["entityId"]
                t_id = generic_rel["target"]["entityId"]

                if generic_rel["direction"] == "directed":
                    key = (s_id, t_id, generic_rel["type"])
                else:
                    key = (tuple(sorted((s_id, t_id))), generic_rel["type"])

                if key not in unique_rels_dict:
                    unique_rels_dict[key] = generic_rel
                    valid_relationships.append(generic_rel)

    print(f"Final total unique relationships to insert: {len(valid_relationships)}")

    if valid_relationships:
        print(f"Attempting to insert relationships into '{relationships_collection.name}' collection...")
        try:
            relationships_collection.drop()
            result = relationships_collection.insert_many(valid_relationships, ordered=False)
            print(f"Successfully inserted {len(result.inserted_ids)} relationships.")
        except pymongo.errors.BulkWriteError as bwe:
            print("Bulk write error during relationship insertion:")
            write_errors = bwe.details.get('writeErrors', [])
            print(f"Total errors: {len(write_errors)}")
            # Show only first 5 errors
            for error_detail in write_errors[:5]:
                print(f"  Index: {error_detail['index']}, Code: {error_detail['code']}, Message: {error_detail['errmsg']}")
            if len(write_errors) > 5:
                print(f"  ... and {len(write_errors) - 5} more errors")
        except Exception as e:
            print(f"An error occurred during relationship insertion: {e}")
else:
    print("No relationships were generated to insert.")

print("\n--- Enhanced Relationship Seeding Complete ---")

Generating enhanced relationships with multi-hop patterns...
Creating ER links for 15 Clear Duplicate sets...
Creating ER links for 8 Subtle Duplicate Clusters...
Creating Enhanced Organizational Structure & UBO links for 5 structures...
Creating enhanced business relationships between organizations...
Creating professional service provider networks...
Creating multi-hop suspicious networks...
Creating extended family and social networks...
Creating financial institution relationships...
Creating employment and board member networks...
Creating Household links for 10 sets...
Creating links involving High-Risk/Watchlisted entities...
Creating some Past/Inactive relationships...
Creating a few generic social/professional links for graph density...
Creating some speculative links for Evolving Risk Individuals...
Creating fourth-hop relationships based on existing network...

Total unique relationships to insert: 519
Final total unique relationships to insert: 519
Attempting to insert rela

In [ ]:
# (At the end of your relationship generation script, or in a separate indexing script)

print("\n--- Attempting to Create Indexes for 'relationships' Collection ---")

try:
    print("Creating indexes for 'relationships' collection...")

    # 1. For finding relationships starting FROM a specific entity (Outbound edges)
    # Often used in $graphLookup's 'connectFromField' or direct queries.
    # Type and active status are common filters.
    relationships_collection.create_index(
        [
            ("source.entityId", pymongo.ASCENDING),
            ("type", pymongo.ASCENDING),
            ("active", pymongo.ASCENDING) # -1 if you more often query for active=true
        ],
        name="rel_source_type_active_idx"
    )
    print("- Index on 'source.entityId', 'type', 'active' created or already exists.")

    # 2. For finding relationships going TO a specific entity (Inbound edges)
    # Often used in $graphLookup's 'connectToField' or direct queries.
    relationships_collection.create_index(
        [
            ("target.entityId", pymongo.ASCENDING),
            ("type", pymongo.ASCENDING),
            ("active", pymongo.ASCENDING) # -1 if you more often query for active=true
        ],
        name="rel_target_type_active_idx"
    )
    print("- Index on 'target.entityId', 'type', 'active' created or already exists.")

    # 3. For querying by relationship type globally
    relationships_collection.create_index([("type", pymongo.ASCENDING)], name="rel_type_idx")
    print("- Index on 'type' created or already exists.")

    # 4. For looking up a specific relationship by its own ID (if you do this)
    # This should be unique.
    relationships_collection.create_index(
        [("relationshipId", pymongo.ASCENDING)],
        name="rel_relationshipId_idx",
        unique=True
    )
    print("- Unique index on 'relationshipId' created or already exists.")

    # 5. Optional: For queries filtering by datasource
    relationships_collection.create_index([("datasource", pymongo.ASCENDING)], name="rel_datasource_idx")
    print("- Index on 'datasource' created or already exists.")

    # 6. Optional: For queries filtering by active status (though often covered by compound indexes above)
    # relationships_collection.create_index([("active", pymongo.ASCENDING)], name="rel_active_idx")
    # print("- Index on 'active' created or already exists.")

    # 7. Optional: If you frequently query for relationships active within a certain time window
    # relationships_collection.create_index(
    #     [
    #         ("active", pymongo.ASCENDING),
    #         ("validFrom", pymongo.ASCENDING),
    #         ("validTo", pymongo.ASCENDING)
    #     ],
    #     name="rel_active_validity_idx",
    #     partialFilterExpression={"active": True} # Index only active relationships with validity dates
    # )
    # print("- Index on 'active', 'validFrom', 'validTo' (partial) created or already exists.")


    print("Relationship index creation process completed.")

except pymongo.errors.OperationFailure as e:
    # Handle specific errors, like "duplicate key error" if unique index fails
    if "duplicate key error" in str(e).lower() and "rel_relationshipId_idx" in str(e).lower():
         print(f"Hint: The unique index on 'relationshipId' failed. Check for duplicate relationshipId values if this was not expected: {e}")
    elif "IndexOptionsConflict" in str(e) or "IndexKeySpecsConflict" in str(e):
        print(f"Index option conflict. An index with similar fields but different options might exist: {e}")
    else:
        print(f"An error occurred during relationship index creation: {e}")
except Exception as e:
    print(f"An unexpected error occurred during relationship index creation: {e}")

# client.close() # If this is the end of this specific script block


--- Attempting to Create Indexes for 'relationships' Collection ---
Creating indexes for 'relationships' collection...
- Index on 'source.entityId', 'type', 'active' created or already exists.
- Index on 'target.entityId', 'type', 'active' created or already exists.
- Index on 'type' created or already exists.
- Unique index on 'relationshipId' created or already exists.
- Index on 'datasource' created or already exists.
Relationship index creation process completed.


# 3. `transactionsv2` Collection

*   **Schema:**
    *   `_id`: `ObjectId()`
    *   `transactionId`: String (e.g., "TXN" + random) - **Unique**
    *   `entityId`: String (ID of your customer involved)
    *   `timestamp`: `ISODate()`
    *   `amount`: Number
    *   `currency`: String (e.g., "USD", "EUR")
    *   `direction`: "incoming", "outgoing"
    *   `counterpartyName`: String
    *   `counterpartyBank`: String (optional)
    *   `counterpartyCountry`: String (2-letter code, important for risk)
    *   `transactionType`: String (e.g., "wire_transfer", "card_payment", "cash_deposit")
    *   `description`: String (optional)
*   **Data Tweaks for `transactions`:**
    1.  Entity J (low risk) - few, small, domestic transactions.
    2.  Entity K (high risk) - large, frequent, international transactions, some to/from high-risk countries.
    3.  Entity L (evolving risk) - initially like J, then add a burst of transactions to a high-risk country.
    4.  Transactions that could be seen as "structuring" (e.g., multiple $9,500 transactions).

In [ ]:
# transactionsv2 Collection Generation
# Creates transactions with fromEntityId and toEntityId for $graphLookup support

import pymongo
from faker import Faker
import random
from datetime import datetime, timedelta, timezone
from bson import ObjectId
import uuid
import numpy as np

# --- Database and Collection Setup ---
transactions_collection = db["transactionsv2"]
entities_collection = db["entities"]

# --- Constants ---
TOTAL_TRANSACTIONS = 15000
BACKGROUND_NOISE_TRANSACTIONS = 12000
SCENARIO_SPECIFIC_TRANSACTIONS = 3000

# Define countries_by_risk
COUNTRIES_BY_RISK_GLOBAL = {
    "low": ["US", "CA", "DE", "FR", "AU", "JP", "GB", "NL", "SE", "NZ"],
    "medium": ["SG", "AE", "CH", "HK", "BR", "IN", "CN", "ZA", "KR", "ES", "IT"],
    "high": ["SY", "KP", "VE", "IR", "AF", "SO", "YE", "LY", "IQ", "MM", "RU", "BY",
             "KY", "PA", "BS", "VU", "MT", "CY", "TC", "SC", "BZ", "LI"]
}

# --- Helper Functions ---
def get_all_entities_for_transactions():
    """Fetches all entities with their key information for transaction generation."""
    entities = list(entities_collection.find(
        {},
        {
            "entityId": 1,
            "entityType": 1,
            "scenarioKey": 1,
            "name.full": 1,
            "riskAssessment.overall.level": 1,
            "addresses": 1,
            "customerInfo.industry": 1,
            "customerInfo.businessType": 1,
            "_id": 0
        }
    ))

    # Add home country to each entity
    for entity in entities:
        home_country = "US"  # Default
        if entity.get("addresses"):
            for addr in entity.get("addresses", []):
                if addr.get("primary") and addr.get("structured", {}).get("country"):
                    home_country = addr["structured"]["country"]
                    break
        entity["homeCountry"] = home_country

    return entities

def get_entities_by_scenario(entities, scenario_pattern):
    """Filter entities by scenario key pattern."""
    return [e for e in entities if scenario_pattern in e.get("scenarioKey", "")]

def get_entity_relationships(entity_id, relationships_collection=None):
    """Get all entities connected to a given entity through relationships."""
    if not relationships_collection:
        relationships_collection = db["relationships"]

    connected_entities = set()

    # Find relationships where entity is source or target
    relationships = list(relationships_collection.find({
        "$or": [
            {"source.entityId": entity_id},
            {"target.entityId": entity_id}
        ],
        "active": True
    }))

    for rel in relationships:
        if rel["source"]["entityId"] == entity_id:
            connected_entities.add(rel["target"]["entityId"])
        else:
            connected_entities.add(rel["source"]["entityId"])

    return list(connected_entities)

def generate_transaction_with_entities(from_entity, to_entity, **kwargs):
    """Generate a transaction between two specific entities."""
    timestamp = kwargs.get("timestamp", fake.date_time_between(start_date="-3y", end_date="now", tzinfo=timezone.utc))

    # Determine transaction characteristics based on entity types and risk levels
    from_risk = from_entity.get("riskAssessment", {}).get("overall", {}).get("level", "low")
    to_risk = to_entity.get("riskAssessment", {}).get("overall", {}).get("level", "low")
    combined_risk = max(from_risk, to_risk, key=lambda x: ["low", "medium", "high"].index(x))

    # Base amount determination
    if kwargs.get("amount"):
        amount = kwargs["amount"]
    elif kwargs.get("is_structuring"):
        amount = round(random.uniform(8000, 9990), 2)
    elif kwargs.get("is_large_value"):
        amount = round(random.uniform(100000, 5000000), 2)
    else:
        if combined_risk == "high":
            amount = round(random.uniform(5000, 250000), 2)
        elif combined_risk == "medium":
            amount = round(random.uniform(1000, 50000), 2)
        else:
            amount = round(random.uniform(50, 10000), 2)

    # Transaction type based on entity types and scenario
    transaction_type = kwargs.get("transaction_type")
    if not transaction_type:
        if from_entity["entityType"] == "organization" and to_entity["entityType"] == "organization":
            transaction_type = random.choice([
                "b2b_wire_transfer", "trade_payment", "intercompany_transfer",
                "investment_transfer", "loan_disbursement", "dividend_payment"
            ])
        elif from_entity["entityType"] == "individual" and to_entity["entityType"] == "individual":
            transaction_type = random.choice([
                "p2p_transfer", "family_support", "personal_loan",
                "gift_transfer", "investment_transfer"
            ])
        else:
            # Mixed individual/organization
            if from_entity["entityType"] == "organization":
                transaction_type = random.choice([
                    "salary_payment", "dividend_payout", "expense_reimbursement",
                    "commission_payment", "loan_disbursement"
                ])
            else:
                transaction_type = random.choice([
                    "investment_deposit", "service_payment", "purchase_payment",
                    "loan_repayment", "subscription_payment"
                ])

    # Payment method
    payment_method = kwargs.get("payment_method")
    if not payment_method:
        if "wire" in transaction_type or amount > 50000:
            payment_method = "SWIFT" if from_entity["homeCountry"] != to_entity["homeCountry"] else "Wire"
        elif "p2p" in transaction_type:
            payment_method = random.choice(["P2PApp", "MobileWallet", "OnlineBanking"])
        elif amount > 10000:
            payment_method = random.choice(["Wire", "ACH", "Check"])
        else:
            payment_method = random.choice(["ACH", "CardNetwork", "OnlineBanking"])

    # Currency - use home country currency or USD for international
    currency = "USD"
    if from_entity["homeCountry"] == to_entity["homeCountry"]:
        currency_map = {"US": "USD", "GB": "GBP", "DE": "EUR", "FR": "EUR", "JP": "JPY", "CA": "CAD", "AU": "AUD"}
        currency = currency_map.get(from_entity["homeCountry"], "USD")

    # Build transaction
    txn = {
        "_id": ObjectId(),
        "transactionId": f"TXN-{uuid.uuid4().hex[:12].upper()}",
        "fromEntityId": from_entity["entityId"],
        "fromEntityType": from_entity["entityType"],
        "fromEntityName": from_entity.get("name", {}).get("full", "Unknown"),
        "toEntityId": to_entity["entityId"],
        "toEntityType": to_entity["entityType"],
        "toEntityName": to_entity.get("name", {}).get("full", "Unknown"),
        "timestamp": timestamp,
        "amount": amount,
        "currency": currency,
        "transactionType": transaction_type,
        "paymentMethod": payment_method,
        "status": kwargs.get("status", "completed" if random.random() > 0.05 else "pending"),
        "channel": kwargs.get("channel", random.choice([
            "online_banking", "mobile_app", "branch", "api", "batch_upload"
        ])),
        "description": kwargs.get("description", fake.sentence(nb_words=random.randint(3, 8))),
        "tags": kwargs.get("tags", []),
        "riskScore": kwargs.get("risk_score", 0),
        "flagged": kwargs.get("flagged", False),
        "additionalDetails": kwargs.get("additional_details", {})
    }

    # Auto-tag based on characteristics
    if amount > 100000:
        txn["tags"].append("large_value")
    if from_entity["homeCountry"] != to_entity["homeCountry"]:
        txn["tags"].append("cross_border")
    if combined_risk == "high":
        txn["tags"].append("high_risk_entity")
    if to_entity["homeCountry"] in COUNTRIES_BY_RISK_GLOBAL["high"]:
        txn["tags"].append("high_risk_jurisdiction")
    if kwargs.get("is_structuring"):
        txn["tags"].append("potential_structuring")

    # Risk scoring
    risk_score = 0
    if "high_risk_entity" in txn["tags"]: risk_score += 30
    if "high_risk_jurisdiction" in txn["tags"]: risk_score += 25
    if "large_value" in txn["tags"]: risk_score += 15
    if "cross_border" in txn["tags"]: risk_score += 10
    if "potential_structuring" in txn["tags"]: risk_score += 40

    txn["riskScore"] = min(risk_score, 100)
    txn["flagged"] = txn["riskScore"] > 60

    return txn

# --- Main Transaction Generation ---
print("Fetching all entities for transaction generation...")
all_entities = get_all_entities_for_transactions()
entities_by_id = {e["entityId"]: e for e in all_entities}

# Categorize entities by scenario
print("Categorizing entities by scenario...")
scenario_entities = {
    "pep": get_entities_by_scenario(all_entities, "pep_individual"),
    "sanctioned_org": get_entities_by_scenario(all_entities, "sanctioned_org"),
    "shell_company": get_entities_by_scenario(all_entities, "shell_company"),
    "hnwi": get_entities_by_scenario(all_entities, "hnwi"),
    "evolving_risk": get_entities_by_scenario(all_entities, "evolving_risk"),
    "household": get_entities_by_scenario(all_entities, "household_set"),
    "complex_org_parent": get_entities_by_scenario(all_entities, "complex_org_parent"),
    "complex_org_sub": get_entities_by_scenario(all_entities, "complex_org_sub"),
    "director": get_entities_by_scenario(all_entities, "director_"),
    "nominee": get_entities_by_scenario(all_entities, "nominee_director")
}

all_transactions = []

# --- 1. Scenario-Specific Transactions (3,000) ---
print(f"Generating {SCENARIO_SPECIFIC_TRANSACTIONS} scenario-specific transactions...")

# 1.1 PEP Transactions (400)
print("  - PEP transactions...")
for pep in scenario_entities["pep"][:10]:  # Top 10 PEPs
    # PEP receiving from shell companies
    for _ in range(3):
        if scenario_entities["shell_company"]:
            shell = random.choice(scenario_entities["shell_company"])
            txn = generate_transaction_with_entities(
                shell, pep,
                amount=round(random.uniform(50000, 500000), 2),
                transaction_type="consulting_fee",
                description="Consulting services rendered",
                tags=["pep_transaction", "shell_to_pep"]
            )
            all_transactions.append(txn)

    # PEP sending to offshore accounts (other entities in high-risk countries)
    high_risk_entities = [e for e in all_entities if e["homeCountry"] in COUNTRIES_BY_RISK_GLOBAL["high"]]
    if high_risk_entities:
        for _ in range(2):
            offshore_entity = random.choice(high_risk_entities)
            txn = generate_transaction_with_entities(
                pep, offshore_entity,
                amount=round(random.uniform(100000, 1000000), 2),
                transaction_type="investment_transfer",
                description="Investment in overseas venture",
                tags=["pep_transaction", "pep_to_offshore"]
            )
            all_transactions.append(txn)

# 1.2 Shell Company Layering (600)
print("  - Shell company layering transactions...")
shell_companies = scenario_entities["shell_company"]
if len(shell_companies) >= 3:
    # Create chains of transactions between shell companies
    for _ in range(100):
        # Pick 3-5 shell companies for a chain
        chain_length = random.randint(3, 5)
        if len(shell_companies) >= chain_length:
            chain = random.sample(shell_companies, chain_length)
            base_amount = round(random.uniform(50000, 500000), 2)

            for i in range(len(chain) - 1):
                # Each hop loses a small percentage (fees)
                amount = base_amount * (0.98 ** i)
                txn = generate_transaction_with_entities(
                    chain[i], chain[i + 1],
                    amount=round(amount, 2),
                    transaction_type="intercompany_transfer",
                    description="Business transfer",
                    tags=["layering", "shell_company_chain"],
                    timestamp=fake.date_time_between(start_date="-6m", end_date="now", tzinfo=timezone.utc)
                )
                all_transactions.append(txn)

# 1.3 Complex Org Structure Transactions (500)
print("  - Complex organization structure transactions...")
for parent in scenario_entities["complex_org_parent"]:
    # Find subsidiaries through relationships
    connected_entities = get_entity_relationships(parent["entityId"])
    subsidiaries = [entities_by_id.get(eid) for eid in connected_entities
                   if entities_by_id.get(eid) and "complex_org_sub" in entities_by_id.get(eid).get("scenarioKey", "")]

    for subsidiary in [s for s in subsidiaries if s]:
        # Regular intercompany transfers
        for _ in range(5):
            direction = random.choice(["parent_to_sub", "sub_to_parent"])
            if direction == "parent_to_sub":
                txn = generate_transaction_with_entities(
                    parent, subsidiary,
                    amount=round(random.uniform(100000, 2000000), 2),
                    transaction_type="capital_injection",
                    description="Subsidiary funding",
                    tags=["intercompany", "parent_subsidiary"]
                )
            else:
                txn = generate_transaction_with_entities(
                    subsidiary, parent,
                    amount=round(random.uniform(50000, 1000000), 2),
                    transaction_type="dividend_payment",
                    description="Quarterly dividend",
                    tags=["intercompany", "subsidiary_parent"]
                )
            all_transactions.append(txn)

# 1.4 Sanctioned Organization Transactions (400)
print("  - Sanctioned organization transactions...")
for sanctioned_org in scenario_entities["sanctioned_org"]:
    # Find intermediaries (other orgs that might unknowingly deal with sanctioned entities)
    intermediary_orgs = [e for e in all_entities
                        if e["entityType"] == "organization"
                        and "sanctioned" not in e.get("scenarioKey", "")
                        and e["entityId"] != sanctioned_org["entityId"]]

    if intermediary_orgs:
        # Create indirect transaction chains
        for _ in range(5):
            intermediary = random.choice(intermediary_orgs)
            final_recipient = random.choice(all_entities)

            # Sanctioned -> Intermediary
            txn1 = generate_transaction_with_entities(
                sanctioned_org, intermediary,
                amount=round(random.uniform(50000, 500000), 2),
                transaction_type="trade_payment",
                description="Equipment purchase",
                tags=["sanctions_evasion_risk", "sanctioned_entity"],
                status="flagged_for_review"
            )
            all_transactions.append(txn1)

            # Intermediary -> Final recipient (1-2 days later)
            txn2 = generate_transaction_with_entities(
                intermediary, final_recipient,
                amount=round(txn1["amount"] * 0.95, 2),  # Slight reduction
                transaction_type="trade_payment",
                description="Resale of goods",
                tags=["potential_sanctions_evasion"],
                timestamp=txn1["timestamp"] + timedelta(days=random.randint(1, 2))
            )
            all_transactions.append(txn2)

# 1.5 HNWI Investment Patterns (400)
print("  - HNWI investment pattern transactions...")
for hnwi in scenario_entities["hnwi"][:10]:
    # Find their service providers through relationships
    connected_entities = get_entity_relationships(hnwi["entityId"])

    # Large investment movements
    investment_targets = [e for e in all_entities
                         if e["entityType"] == "organization"
                         and "investment" in e.get("customerInfo", {}).get("industry", "").lower()]

    if investment_targets:
        for _ in range(4):
            target = random.choice(investment_targets)
            txn = generate_transaction_with_entities(
                hnwi, target,
                amount=round(random.uniform(500000, 5000000), 2),
                transaction_type="investment_deposit",
                description="Portfolio investment",
                tags=["hnwi_transaction", "large_investment"]
            )
            all_transactions.append(txn)

# 1.6 Evolving Risk Pattern Transactions (300)
print("  - Evolving risk pattern transactions...")
for evolving_entity in scenario_entities["evolving_risk"]:
    # Create a pattern that shows risk evolution
    # Early transactions: normal
    for _ in range(5):
        counterparty = random.choice([e for e in all_entities if e["homeCountry"] == evolving_entity["homeCountry"]])
        txn = generate_transaction_with_entities(
            evolving_entity, counterparty,
            amount=round(random.uniform(500, 5000), 2),
            timestamp=fake.date_time_between(start_date="-3y", end_date="-1y", tzinfo=timezone.utc),
            tags=["normal_pattern"]
        )
        all_transactions.append(txn)

    # Recent transactions: suspicious
    high_risk_counterparties = [e for e in all_entities
                               if e.get("riskAssessment", {}).get("overall", {}).get("level") == "high"
                               or e["homeCountry"] in COUNTRIES_BY_RISK_GLOBAL["high"]]

    if high_risk_counterparties:
        for _ in range(5):
            counterparty = random.choice(high_risk_counterparties)
            txn = generate_transaction_with_entities(
                evolving_entity, counterparty,
                amount=round(random.uniform(10000, 100000), 2),
                timestamp=fake.date_time_between(start_date="-3m", end_date="now", tzinfo=timezone.utc),
                tags=["pattern_change", "risk_escalation"],
                flagged=True
            )
            all_transactions.append(txn)

# 1.7 Household Member Transactions (200)
print("  - Household member transactions...")
household_members = scenario_entities["household"]
for i in range(0, len(household_members) - 1, 2):
    if i + 1 < len(household_members):
        member1 = household_members[i]
        member2 = household_members[i + 1]

        # Regular transfers between household members
        for _ in range(10):
            sender, receiver = random.choice([(member1, member2), (member2, member1)])
            txn = generate_transaction_with_entities(
                sender, receiver,
                amount=round(random.uniform(100, 10000), 2),
                transaction_type=random.choice(["family_support", "household_expense", "gift_transfer"]),
                tags=["household_transfer"]
            )
            all_transactions.append(txn)

# 1.8 Structuring Patterns (200)
print("  - Structuring pattern transactions...")
# Pick random entities to perform structuring
structuring_entities = random.sample(all_entities, min(20, len(all_entities)))
for entity in structuring_entities[:10]:
    # Create a structuring pattern
    target_entity = random.choice([e for e in all_entities if e["entityId"] != entity["entityId"]])
    base_timestamp = fake.date_time_between(start_date="-1m", end_date="now", tzinfo=timezone.utc)

    # Multiple transactions just under reporting threshold
    num_structured = random.randint(3, 6)
    for j in range(num_structured):
        txn = generate_transaction_with_entities(
            entity, target_entity,
            amount=round(random.uniform(9000, 9900), 2),
            timestamp=base_timestamp + timedelta(hours=j * random.randint(2, 8)),
            is_structuring=True,
            description="Cash deposit",
            tags=["potential_structuring", "below_threshold"]
        )
        all_transactions.append(txn)

# --- 2. Background Noise Transactions (12,000) ---
print(f"Generating {BACKGROUND_NOISE_TRANSACTIONS} background noise transactions...")

# Get all individual and organization entities for background transactions
individuals = [e for e in all_entities if e["entityType"] == "individual"]
organizations = [e for e in all_entities if e["entityType"] == "organization"]

for i in range(BACKGROUND_NOISE_TRANSACTIONS):
    if i % 1000 == 0:
        print(f"  - Generated {i}/{BACKGROUND_NOISE_TRANSACTIONS} background transactions...")

    # Randomly select transaction pattern
    pattern = random.choices(
        ["individual_to_individual", "individual_to_org", "org_to_individual", "org_to_org"],
        weights=[0.15, 0.35, 0.25, 0.25],
        k=1
    )[0]

    # Select entities based on pattern
    if pattern == "individual_to_individual" and len(individuals) >= 2:
        from_entity, to_entity = random.sample(individuals, 2)
    elif pattern == "individual_to_org" and individuals and organizations:
        from_entity = random.choice(individuals)
        to_entity = random.choice(organizations)
    elif pattern == "org_to_individual" and organizations and individuals:
        from_entity = random.choice(organizations)
        to_entity = random.choice(individuals)
    elif pattern == "org_to_org" and len(organizations) >= 2:
        from_entity, to_entity = random.sample(organizations, 2)
    else:
        # Fallback to any two entities
        if len(all_entities) >= 2:
            from_entity, to_entity = random.sample(all_entities, 2)
        else:
            continue

    # Generate normal transaction
    txn = generate_transaction_with_entities(
        from_entity, to_entity,
        timestamp=fake.date_time_between(start_date="-3y", end_date="now", tzinfo=timezone.utc),
        tags=["background_noise"]
    )
    all_transactions.append(txn)

# --- 3. Insert Transactions ---
print(f"\nTotal transactions generated: {len(all_transactions)}")

if all_transactions:
    print(f"Attempting to insert transactions into '{transactions_collection.name}' collection...")
    try:
        transactions_collection.drop()
        result = transactions_collection.insert_many(all_transactions, ordered=False)
        print(f"Successfully inserted {len(result.inserted_ids)} transactions.")
    except pymongo.errors.BulkWriteError as bwe:
        print(f"Bulk write error during transaction insertion:")
        write_errors = bwe.details.get('writeErrors', [])
        print(f"Total errors: {len(write_errors)}")
        for error_detail in write_errors[:5]:
            print(f"  Index: {error_detail['index']}, Code: {error_detail['code']}, Message: {error_detail['errmsg']}")
    except Exception as e:
        print(f"An error occurred during transaction insertion: {e}")

# --- 4. Create Indexes for Graph Lookups ---
print("\n--- Creating indexes for graph lookups on transactions ---")
try:
    # Primary indexes for graph traversal
    transactions_collection.create_index(
        [("fromEntityId", pymongo.ASCENDING), ("timestamp", pymongo.DESCENDING)],
        name="txn_from_entity_time_idx"
    )
    print("- Index on 'fromEntityId' and 'timestamp' created.")

    transactions_collection.create_index(
        [("toEntityId", pymongo.ASCENDING), ("timestamp", pymongo.DESCENDING)],
        name="txn_to_entity_time_idx"
    )
    print("- Index on 'toEntityId' and 'timestamp' created.")

    # Compound index for bidirectional lookups
    transactions_collection.create_index(
        [("fromEntityId", pymongo.ASCENDING), ("toEntityId", pymongo.ASCENDING)],
        name="txn_from_to_idx"
    )
    print("- Index on 'fromEntityId' and 'toEntityId' created.")

    # Other useful indexes
    transactions_collection.create_index([("transactionId", pymongo.ASCENDING)], unique=True, name="txn_id_unique_idx")
    transactions_collection.create_index([("timestamp", pymongo.DESCENDING)], name="txn_timestamp_idx")
    transactions_collection.create_index([("amount", pymongo.DESCENDING)], name="txn_amount_idx")
    transactions_collection.create_index([("riskScore", pymongo.DESCENDING)], name="txn_risk_idx")
    transactions_collection.create_index([("tags", pymongo.ASCENDING)], name="txn_tags_idx")
    transactions_collection.create_index([("flagged", pymongo.ASCENDING)], name="txn_flagged_idx")

    print("- Additional indexes created successfully.")

except Exception as e:
    print(f"Error creating indexes: {e}")

print("\n--- Enhanced transaction generation complete ---")
print(f"""
Summary:
- Total transactions: {len(all_transactions)}
- Background noise: {sum(1 for t in all_transactions if "background_noise" in t.get("tags", []))}
- Scenario-specific: {sum(1 for t in all_transactions if "background_noise" not in t.get("tags", []))}
- Flagged transactions: {sum(1 for t in all_transactions if t.get("flagged", False))}

The transactions now support MongoDB $graphLookup with:
- fromEntityId: References the source entity
- toEntityId: References the target entity
- Both fields link to actual entities in the entities collection
- Proper indexes for efficient graph traversal
""")

Fetching all entities for transaction generation...
Categorizing entities by scenario...
Generating 3000 scenario-specific transactions...
  - PEP transactions...
  - Shell company layering transactions...
  - Complex organization structure transactions...
  - Sanctioned organization transactions...
  - HNWI investment pattern transactions...
  - Evolving risk pattern transactions...
  - Household member transactions...
  - Structuring pattern transactions...
Generating 12000 background noise transactions...
  - Generated 0/12000 background transactions...
  - Generated 1000/12000 background transactions...
  - Generated 2000/12000 background transactions...
  - Generated 3000/12000 background transactions...
  - Generated 4000/12000 background transactions...
  - Generated 5000/12000 background transactions...
  - Generated 6000/12000 background transactions...
  - Generated 7000/12000 background transactions...
  - Generated 8000/12000 background transactions...
  - Generated 9000/12